# Conceptual Pacts through small language models

In [1]:
#!pip install google_trans_new
!pip install -U deep-translator
!pip install scikit-learn
!pip install sciPy
!pip install nltk

In [2]:
import sys
sys.path.append("../python/")
import pentoref.IO as IO
import sqlite3 as sqlite
#from google_trans_new import google_translator
from deep_translator import GoogleTranslator
import datetime

import sklearn
from sklearn.feature_extraction import DictVectorizer
#from sklearn.preprocessing import LabelEncoder
#l = LabelEncoder()

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, classification_report

from nltk.classify import SklearnClassifier
#nltk.download('punkt')  # if using stemming in German

from random import shuffle
import numpy as np
import scipy.stats as stats
from scipy import optimize
import pandas as pd
from math import log
import random
import copy

from collections import Counter
import matplotlib.pyplot as plt

from pentoref.IOutils import clean_utt
from machine_learning_utils import calculate_mcnemar_test

In [3]:
def remove_reparanda(utt):
    "removes the content between ( and + though leaves those intact"
    repair_depth = 0
    cleaned_utt = ""
    for c in utt:
        if c == "+":
            repair_depth-=1
        elif c == "(":
            repair_depth+=1
        elif repair_depth>0:
            continue
        cleaned_utt+=c
    assert repair_depth==0, "repair depth not 0:" + utt
    return cleaned_utt
clean_utt(remove_reparanda("(( hello + hello) + {F um } hello)"))
          

'hello'

In [4]:
# Create databases if required
if False:   # make True if you need to create the databases from the derived data
    for corpus_name in ["TAKE", "TAKECV", "PENTOCV"]:
        data_dir = "../../../pentoref/{0}_PENTOREF".format(corpus_name)
        dfwords, dfutts, dfrefs, dfscenes, dfactions = IO.convert_subcorpus_raw_data_to_dataframes(data_dir)
        IO.write_corpus_to_database("{0}.db".format(corpus_name),
                                    corpus_name, dfwords, dfutts, dfrefs, dfscenes, dfactions)

In [5]:
# Connect to database
CORPUS = "PENTOCV"
db = sqlite.connect("{0}.db".format(CORPUS))
cursor = db.cursor()
# get the table column header names
print("utts", [x[1] for x in cursor.execute("PRAGMA table_info(utts)")])
print("words", [x[1] for x in cursor.execute("PRAGMA table_info(words)")])
print("refs", [x[1] for x in cursor.execute("PRAGMA table_info(refs)")])
print("scenes", [x[1] for x in cursor.execute("PRAGMA table_info(scenes)")])
print("actions", [x[1] for x in cursor.execute("PRAGMA table_info(actions)")])

utts ['gameID', 'uttID', 'starttime', 'endtime', 'utt', 'utt_clean', 'role', 'speaker']
words ['gameID', 'uttID', 'position', 'word', 'lemma', 'tag']
refs ['refID', 'gameID', 'uttID', 'text', 'id', 'piece', 'location']
scenes ['timestampID', 'gameID', 'pieceID', 'position_global', 'position_x', 'position_y', 'shape', 'shape_distribution', 'shape_orientation', 'shape_skewness_horizontal', 'shape_skewness_vertical', 'shape_edges', 'colour', 'colour_distribution', 'colour_hsv', 'colour_rgb']
actions ['gameID', 'starttime', 'endtime', 'hand', 'action', 'piece']


## Get utterances from certain time periods in each experiment or for certain episodes

In [6]:
if False:
    translator = GoogleTranslator(source='de', target='en')

    for row in db.execute("SELECT gameID, starttime, speaker, utt_clean, utt FROM utts" + \
                       # " WHERE starttime >= 200 AND starttime <= 300" + \
                         ' WHERE gameID = "r1_1_1_b"' + \
                        " ORDER BY gameID, starttime"):
        print(row)
        line = row[3]
        print(line)
        if not line:
            continue
        translate_text = translator.translate(line,lang_src='de',lang_tgt='en') 
        print(translate_text)

## Build dataset

In [7]:
# just focus on single pieces, not references to multiple pieces
good_pieces = ["X", "Y", "P", "N", "U", "F", "Z", "L", "T", "I", "W", "V"]

In [8]:
references_per_pair = {}  # all data will be stored here with keys =pairnum
for row in db.execute("SELECT id, gameID, text, uttID FROM refs" + \
#for row in db.execute("SELECT shape, colour, orientation, gridPosition, gameID, pieceID FROM scenes" + \
                     " ORDER by gameID"):
    if False: 
        # TAKE
        isTarget = db.execute('SELECT refID FROM refs WHERE gameID ="' + row[4] + '" AND pieceID ="' + row[5] + '"')
        target = False 
        for r1 in isTarget:
            target = True
        if not target:
            continue

    #TAKE
    #shape, colour, orientation, gridPosition, gameID, pieceID = row
    #piece = colour  #+ "_" + shape #shape + "_" + colour
    
    #PENTOCV
    piece, gameID, text, uttID = row
    
    # some manual corrections of disfluencies
    error_found = False
    if "(der + (das + das + das) grüne $m)" in text.lower():
        error_found = True
        text = "(der + (das + (das + das ))) grüne $m)"
    elif "(der {f äh:m:} + der) + der) winkel" in text.lower():
        error_found = True
        text = "((der {f äh:m:} + der) + der) winkel"
    elif "das {br- + blaue} lange" in text.lower():
        error_found = True
        text = "das (br- + blaue) Lange"
    elif "das: ({f äh} ja .) andere blaue $z" in text.lower():
        error_found = True
        text = "das: ({f äh} ja . + ) andere blaue $z"
    elif "den (ist das rosa oben) block" in text.lower():
        error_found = True
        text = "den (ist das rosa oben + ) block"
    elif """(<p="dieses">die-</p> (genau) + das) element""" in text.lower():
        error_found = True
        text = """(<p="dieses">die-</p> (genau  +) + das) element"""
    elif """das $t (dieser blaue <p="senkrecht">senk-</p>""" in text.lower():
        error_found = True
        text = """das $t (dieser blaue <p="senkrecht">senk-</p>+)"""
    elif """es">wie's</v> jetzt <v="ist">is'</v> und steckst <ref id="f" piece="target"> es""" in text.lower():
        error_found = True
        text = """wie's jetzt <v="ist">is'</v> und steckst <ref id="f" piece="target"> es"""
    elif """es">wie's</v>""" in text.lower():
        error_found = True
        text = """wie's"""
        
    if error_found:
        print("replacing with", text, "at", gameID, uttID)
        
        
    
    clean_text = clean_utt(remove_reparanda(text.lower()))
    assert clean_text!="", count
    
    
    pair_num = gameID.split("_")[0]
    # get speaker
    speaker = list(db.execute('SELECT speaker FROM utts WHERE uttID =' + str(uttID)))[0][0]
    speaker = pair_num + "_" + speaker
    
    end_time =  float(list(db.execute('SELECT endtime FROM utts WHERE uttID =' + str(uttID)))[0][0])
    
    #if not pair_num == PAIR_NUM:
    #    continue
    
        
    if piece not in good_pieces:
        continue
        
    if not references_per_pair.get(pair_num):
        references_per_pair[pair_num] = []
    
    
    if "_s" in gameID:
        continue # just get the build phases for now due to inconsistent labelling

    references_per_pair[pair_num].append((clean_text, speaker, uttID, end_time, piece))


    # sort by end time
for pair_num in references_per_pair.keys():
    ref_list = references_per_pair[pair_num]
    ref_list = sorted(ref_list, key=lambda x:x[3])
    references_per_pair[pair_num] = ref_list

print(references_per_pair.keys())
print(references_per_pair['r3'][0:20])

replacing with wie's jetzt <v="ist">is'</v> und steckst <ref id="f" piece="target"> es at r2_1_1_b 13355
replacing with wie's at r2_1_4_b 13568
replacing with (der + (das + (das + das ))) grüne $m) at r4_1_3_b 10823
replacing with ((der {f äh:m:} + der) + der) winkel at r4_1_3_b 10880
replacing with das (br- + blaue) Lange at r7_1_1_b 1215
replacing with das: ({f äh} ja . + ) andere blaue $z at r8_1_3_s 2454
replacing with den (ist das rosa oben + ) block at r8_1_5_b 4428
replacing with (<p="dieses">die-</p> (genau  +) + das) element at r8_1_8_b 4919
replacing with das $t (dieser blaue <p="senkrecht">senk-</p>+) at r8_2_17_b 3696
dict_keys(['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8'])
[('gelben stein', 'r3_B', 8886, 129.257, 'U'), ('das rote kreuz', 'r3_B', 8893, 141.68, 'X'), ('daran', 'r3_B', 8897, 148.215, 'U'), ('der orange stein', 'r3_B', 8897, 148.215, 'L'), ('das l', 'r3_A', 7900, 151.582, 'L'), ('daran', 'r3_B', 8902, 157.211, 'X'), ('das grüne t', 'r3_B', 8902, 157.211, 'T

# Create language models and language model features

In [9]:
# null hyp 1: the referring expression is no less predictable given the other mentions for a given piece at a given time point
# regardless of order (so like a language model, all mentions are equally likely and there's no correlation with time)
# of how p(ref|previous) =< p(ref|all)


In [10]:
# ASSUMPTION: only dealing with full names, not anaphors # TODO for this paper may actually leave them in
if True: # False TODO for this paper may actually leave them in
    anaphors = ["es", "das", 'er', "da", "der", "ihn", "den", "sie", "die", "damit", "daran", "dem"]
    for pair_num in references_per_pair.keys():
        refs = references_per_pair[pair_num]
        references_per_pair[pair_num] = list(filter(lambda x:x[0] not in anaphors, refs))


In [11]:
# Get stats for whole corpus and pieces
piece_counter = Counter()
for pair_num in references_per_pair.keys():
    list_refs = [ref[4] for ref in references_per_pair[pair_num]]
    piece_counter.update(list_refs) 
print("TOTAL REFERENCES", piece_counter.total())
sorted(piece_counter.items())

TOTAL REFERENCES 1899


[('F', 134),
 ('I', 139),
 ('L', 161),
 ('N', 197),
 ('P', 147),
 ('T', 191),
 ('U', 144),
 ('V', 162),
 ('W', 170),
 ('X', 177),
 ('Y', 162),
 ('Z', 115)]

In [12]:
# let's have a look at how one pair talks about a piece over time
for pair_num in references_per_pair.keys():
    print(pair_num, len(references_per_pair[pair_num]), "refs")
mean_length =np.mean([len(v) for k,v in references_per_pair.items()])
print(mean_length)
# select test as closest to mean length, heldout as second closest
for pair_num in references_per_pair.keys():
    print(pair_num, abs(len(references_per_pair[pair_num])-mean_length))

r1 100 refs
r2 170 refs
r3 265 refs
r4 285 refs
r5 143 refs
r6 258 refs
r7 352 refs
r8 326 refs
237.375
r1 137.375
r2 67.375
r3 27.625
r4 47.625
r5 94.375
r6 20.625
r7 114.625
r8 88.625


In [13]:
TEST = 'r6'
HELDOUT = 'r3'
EXTRA_HELDOUT = 'r7' # note this should probably be r4, in principle shouldn't affect results

In [14]:
# let's have a look at how one pair talks about a piece over time
for ref in filter(lambda x:x[4] == 'Z', references_per_pair['r6']):
    print(ref)

('das türkise', 'r6_B', 7002, 553.312, 'Z')
('das türkise', 'r6_B', 7004, 556.626, 'Z')
('das türkise', 'r6_B', 7185, 824.143, 'Z')
('das türkise', 'r6_B', 7250, 937.942, 'Z')
('das türkise', 'r6_B', 7260, 953.935, 'Z')
('das türkise', 'r6_B', 7408, 1251.277, 'Z')
('das blaue', 'r6_B', 7436, 1302.236, 'Z')
('das türkise', 'r6_B', 7834, 2017.387, 'Z')


In [15]:
class mini_language_model():
    
    def __init__(self, order, smoothing_k=0.0001):
        self.order = order
        self.ngrams = {}
        self.num_training_sents = 0
        self.smoothing_k = smoothing_k
        for o in range(1, order+1):
            self.ngrams[o] = {}
        self.unigram_denom = 0
        self.vocab_size = 0
    
    
    def add_ngram_to_model(self, ngram, order):
        ngram_text = "_".join(ngram)
        if not self.ngrams.get(order).get(ngram_text):
            self.ngrams[order][ngram_text] = 0
        self.ngrams[order][ngram_text]+=1

        
    def minus_ngram_from_model(self, ngram, order):
        ngram_text = "_".join(ngram)
        if not self.ngrams.get(order).get(ngram_text):
            self.ngrams[order][ngram_text] = 0
            
        self.ngrams[order][ngram_text]-=1
      
    
    def add_counts_from_other_model(self, other_lm):
        assert(other_lm.order == self.order)
        for n in range(1, self.order+1):
            #print("ngrams before", self.ngrams)
            combined_keys = list(self.ngrams[n].keys() | other_lm.ngrams[n].keys())
            #print("other after", self.ngrams)
            for key in combined_keys:
                
                count_self = self.ngrams[n].get(key)
                if not count_self:
                    count_self = 0
                count_other = other_lm.ngrams[n].get(key)
                if not count_other:
                    count_other = 0
                #print(key, count_self, count_other)
                self.ngrams[n][key] = count_self + count_other
        self.unigram_denom = sum(self.ngrams[1].values())
        self.vocab_size = len(self.ngrams[1])
        #print("update")
        #print(self.unigram_denom, "unigram counts")
        #print(self.vocab_size, "vocab size")
        
    def train(self, sents):
        for sent in sents:
            padded = ["<s>"] * (self.order -1) + sent + ["</s>"]
            for i in range(self.order-1, len(padded)):
                for n in range(1, self.order+1):
                    target = padded[i]
                    context = padded[i-(n-1):i]
                    self.add_ngram_to_model(context + [target], n)
                    if n> 1:
                        self.add_ngram_to_model(context, n-1)
                    
            self.num_training_sents += 1
        #print(self.ngrams)
        self.unigram_denom = sum(self.ngrams[1].values())
        self.vocab_size = len(self.ngrams[1])
        #print(self.unigram_denom, "unigram counts")
        #print(self.vocab_size, "vocab size")
        
    def de_train(self, sents):
        # take away these counts
        for sent in sents:
            padded = ["<s>"] * (self.order -1) + sent + ["</s>"]
            for i in range(self.order-1, len(padded)):
                for n in range(1, self.order+1):
                    target = padded[i]
                    context = padded[i-(n-1):i]
                    self.minus_ngram_from_model(context + [target], n)
        #print(self.ngrams)
        self.unigram_denom = sum(self.ngrams[1].values())
        self.vocab_size = len(self.ngrams[1])
        #print(self.unigram_denom, "unigram counts")
        #print(self.vocab_size, "vocab size")
        
    def prob_lidstone(self, ngram, order):
        """Add-k smoothing using the discount parameter for this model."""
        ngram_text =  "_".join(ngram)
        ngram_count = self.ngrams[order].get(ngram_text)
        if not ngram_count:
            ngram_count = 0
        num = ngram_count + self.smoothing_k
        if order == 1:
            denom = self.unigram_denom + (self.smoothing_k * self.vocab_size)
            if self.unigram_denom == 0:
                #print("warning no training, returning k/40/10")
                denom = 150 + (self.smoothing_k * 15)
            #print(denom)
        else:
            context = ngram[:-1]
            context_text = "_".join(context)
            #print(context_text)
            context_count = self.ngrams[order-1].get(context_text)
            if not context_count:
                context_count = 0
            #print(context_count)
            all_contexts = len(self.ngrams[order-1].keys())
            denom = context_count + (self.smoothing_k * all_contexts)
            if self.unigram_denom == 0 or context_count == 0:
                #print("warning no training, returning k/40/10")
                denom = 40 + (self.smoothing_k * 10)
            #print(denom)
        assert num/denom <=1 and num/denom >0, str(num) + " " + str(denom) + " " + str(num/denom) + " " + str(ngram) + " n=" + str(order) + " k=" + str(self.smoothing_k)
        return num/denom
    

In [16]:
m = mini_language_model(3, smoothing_k=0.1)
m.train([["I", "like", "bill"], ["I", "like", "mary"]])
print(m.ngrams)
print(m.prob_lidstone(["I", "like", "mary"], 3))

m2 = mini_language_model(3)
m2.train([["I", "like", "bill", 'today'], ["I", "like", "mary"]])
print(m2.ngrams)
m.add_counts_from_other_model(m2)
print(m.ngrams)

{1: {'I': 4, '<s>': 2, 'like': 4, 'bill': 2, '</s>': 2, 'mary': 2}, 2: {'<s>_I': 4, '<s>_<s>': 2, 'I_like': 4, 'like_bill': 2, 'bill_</s>': 1, 'like_mary': 2, 'mary_</s>': 1}, 3: {'<s>_<s>_I': 2, '<s>_I_like': 2, 'I_like_bill': 1, 'like_bill_</s>': 1, 'I_like_mary': 1, 'like_mary_</s>': 1}}
0.23404255319148937
{1: {'I': 4, '<s>': 2, 'like': 4, 'bill': 2, 'today': 2, '</s>': 2, 'mary': 2}, 2: {'<s>_I': 4, '<s>_<s>': 2, 'I_like': 4, 'like_bill': 2, 'bill_today': 2, 'today_</s>': 1, 'like_mary': 2, 'mary_</s>': 1}, 3: {'<s>_<s>_I': 2, '<s>_I_like': 2, 'I_like_bill': 1, 'like_bill_today': 1, 'bill_today_</s>': 1, 'I_like_mary': 1, 'like_mary_</s>': 1}}
{1: {'I': 8, '<s>': 4, 'like': 8, 'bill': 4, '</s>': 4, 'mary': 4, 'today': 2}, 2: {'<s>_I': 8, '<s>_<s>': 4, 'I_like': 8, 'like_bill': 4, 'bill_</s>': 1, 'like_mary': 4, 'mary_</s>': 2, 'bill_today': 2, 'today_</s>': 1}, 3: {'<s>_<s>_I': 4, '<s>_I_like': 4, 'I_like_bill': 2, 'like_bill_</s>': 1, 'I_like_mary': 2, 'like_mary_</s>': 2, 'bill_

In [17]:
def get_refs_in_timerange_for_piece(pair_num, piece, time_uttend_start, time_uttend_end, references_per_pair):
    """Returns a list of the refs from a certain time start to a time end in order for a given piece.
    NOTE FOR NOW refs in the same utterance are either all included or all excluded from this method"""
    if not references_per_pair.get(pair_num):
        return []
    return list(filter(lambda x:(x[3]>=time_uttend_start and x[3]<time_uttend_end) and x[4]==piece,
                       references_per_pair[pair_num]))
    

In [18]:
get_refs_in_timerange_for_piece('r3', 'L', 0, 687.481, references_per_pair)

[('der orange stein', 'r3_B', 8897, 148.215, 'L'),
 ('das l', 'r3_A', 7900, 151.582, 'L'),
 ('der orange winkel', 'r3_B', 9019, 374.137, 'L'),
 ('der orange winkel', 'r3_B', 9042, 477.238, 'L')]

In [19]:
# Test 1: Is there an increasing similarity to the previous references as the dialogue progresses?
# Test 2: does a reference have higher average similarity to other references in the same dialogue compared to those in different dialogyes?

# Create reference resolution training + test data with features from the model
* TODO setting - exclude and re-compute based on pieces 'in play', or assume the robot doesn't know - try both
* Try 1) global model only without active learning, 2) global + local standard 3) global and local incrementally/dynamically updating, see the difference in ref res. Measures on accuracy + surprisal.
* Naive bayes with individual language models for each piece + marginalization? OR model can work independently with any probabilistic classifier as a joint probability?
* Get an optimal weighting between the local, incremental model and global models

In [20]:
def get_rank_dict_for_prob_dict(prob_dict):
    rank_dict = {}
    rank = 0
    for item in sorted(prob_dict.items(), key=lambda x:x[1]):
        rank+=1
        rank_dict[item[0]] = rank
    return rank_dict
get_rank_dict_for_prob_dict({"K": 0.34, "Y": 0.35, "X": 0.33})

{'X': 1, 'K': 2, 'Y': 3}

In [21]:
def moving_average(a_list):
    averages = []
    total = 0
    for i, a in enumerate(a_list):
        total+=a
        averages.append(total/(i+1))
    return averages

In [22]:
def mean_cross_entropy_on_test_folds(k, pair_piece_lms_orig, folds, prob_model):
    """Returns the mean of the cross entropy on the folds using the appropriate lm.
    Assumes folds will be in correct order w.r.t. time of reference."""
    # firstly, duplicate the language model to avoid any side effects
    #print(k)
    pair_piece_lms = copy.deepcopy(pair_piece_lms_orig)
    # Set the k for all to be the same
    for pair_num in pair_piece_lms.keys():
        for piece in pair_piece_lms[pair_num].keys():
            pair_piece_lms[pair_num][piece].smoothing_k = k[0]
            
    cross_entropies = []  # get the cross-entropy for each fold
    for pair_num in folds.keys():
        test_set = folds[pair_num]
        s = 0
        count = 0
        for i, ref_info in enumerate(test_set):
            # assume we know the correct ref piece
            piece = ref_info[4]
            lm = pair_piece_lms[pair_num][piece]
            ref = ["<s>"] * (lm.order -1) + ref_info[0].split() + ["</s>"]
            prob = 0
            for j in range(lm.order-1, len(ref)):
                target = ref[j]
                context = ref[j-(lm.order-1):j]
                ngram = context + [target]
                prob += log(lm.prob_lidstone(ngram, lm.order))
            s += prob
            count +=1
            if prob_model == 'self':
                # self language model, starting empty for a piece
                # then will train as it encounters each piece
                #print(pair_piece_lms[pair_num][piece].num_training_sents, "train sents so far before")
                pair_piece_lms[pair_num][piece].train([ref_info[0].split()])
    
        cross_entropy = -s / count
        cross_entropies.append(cross_entropy)
    #print(k, cross_entropies, np.mean(cross_entropies))
    return np.mean(cross_entropies)

In [23]:
training_folds = {k:references_per_pair[k] for k in filter(lambda x:x!=TEST, references_per_pair.keys())}
print(training_folds.keys())
print(len(training_folds['r1']), training_folds['r1'][:10])

dict_keys(['r1', 'r2', 'r3', 'r4', 'r5', 'r7', 'r8'])
100 [('das orange l', 'r1_B', 5887, 204.521, 'L'), ('das kreuz', 'r1_B', 5893, 219.022, 'X'), ('das grüne t', 'r1_B', 5901, 228.117, 'T'), ('das t', 'r1_B', 5903, 231.474, 'T'), ('das t', 'r1_B', 5907, 241.893, 'T'), ('das t', 'r1_B', 5907, 241.893, 'T'), ('das gelbe', 'r1_B', 5919, 258.434, 'U'), ('die brücke', 'r1_B', 5921, 260.36, 'U'), ('das pinkfarbene teil', 'r1_B', 5924, 266.23, 'P'), ('den blauen balken', 'r1_B', 5935, 304.366, 'I')]


In [24]:
def get_params_for_lowest_entropy_on_fold_test(folds, pieces, n_range, k_val_range, prob_model='self'):
    """An x-fold process run multiple times to find the best params of n and k (smoothing constant)
    for an ngram model
    """
    results = {}  # store results for each model
    for test_n in n_range:
        pair_piece_lms = {}  # will produce all lms for each speaker for each piece with order test_n
        for test_pair in folds.keys():
            # test pair is the list of refs to get the entropy results on in both cases
            # in 'other' model, referent_lm model is trained on refs on all other folds
            # in 'self' model, referent_lm model is trained on test_pair incrementally dynamically
                  #i.e. retrained with the ref added to the counts after that ref is tested
            pair_piece_lms[test_pair] = {}  # key piece, value language model for that piece
            for piece in pieces:
                if prob_model == 'other':
                    # train the model (smoothing can be changed as it doesn't affect training)

                    training_refs = []
                    
                    for other_pair_num in folds.keys():
                        if other_pair_num == test_pair:
                            continue
                        training_refs.extend([ref for ref in filter(lambda x:x[4]==piece, folds[other_pair_num])])  
                    training_fold = [r[0] for r in training_refs]
                    #print("length of training fold for lm2", piece, test_pair, len(training_fold))
                    if len(training_fold) <1:
                        print("not enough data", piece, pair_num)
                        continue
                    pair_piece_lms[test_pair][piece] = mini_language_model(test_n, smoothing_k=0.0001)    
                    pair_piece_lms[test_pair][piece].train([sent.split() for sent in training_fold])

                elif prob_model == 'self':
                    # just initialize lm for piece, no need to train
                    pair_piece_lms[test_pair][piece] = mini_language_model(test_n, smoothing_k=0.0001)    
             
            
        # run nelder mead to get best k for this n
        best = optimize.minimize(
                mean_cross_entropy_on_test_folds,
                k_val_range[0],    # first argument that to be optimized (k)
                args=(pair_piece_lms, folds, prob_model),  # other arguments to the function
                method='Nelder-Mead', # use nelder mead to find minima
                tol=0.0001, # to this degree of error
                options={'disp': False},
                bounds=[k_val_range]
        )
        best_k, best_ce = best.x[0], best.fun
        print("best k", best_k, best_ce)
        results[test_n] = (best_k, best_ce)
    
    print(results)
    n = min(results.items(), key=lambda x:x[1][1])[0]
    k = results[n][0]
               
    return n, k, results

In [25]:
# Firstly, in a self-supervised way, optimize the global + local language models in terms of n and smoothing param to get the lowest cross-entropy
# We only go for the global best average across all pairs and pieces for the rest of the experiments
print(TEST, HELDOUT)
opt_params = {}
n_range = [1,2,3]
k_val_range = [0.00000001, 1]

l = list(filter(lambda x:x!=TEST, references_per_pair.keys()))

all_n_self = []
all_k_self = []
all_n_other = []
all_k_other = []

for heldout in l:
    training_folds = {k:references_per_pair[k] for k in filter(lambda x:x not in [TEST, heldout], references_per_pair.keys())}
    
    n_self, k_self, lm_opt_results_self = get_params_for_lowest_entropy_on_fold_test(training_folds, good_pieces,
                                                                  n_range, k_val_range, prob_model='self')


    all_n_self.append(n_self)
    all_k_self.append(k_self)

    n_other, k_other, lm_opt_results_other = get_params_for_lowest_entropy_on_fold_test(training_folds, good_pieces,
                                                                  n_range, k_val_range, prob_model='other')
    
    all_n_other.append(n_other)
    all_k_other.append(k_other)
    
    
opt_params["n_self"] = round(np.mean(all_n_self))  # NB python rounds down 0.5 but shouldn't matter
opt_params["k_self"] = np.mean(all_k_self)

opt_params["n_other"] = round(np.mean(all_n_other)) # NB python rounds down 0.5 but shouldn't matter
opt_params["k_other"] = np.mean(all_k_other)
print(opt_params)

r6 r3
best k 1.0 6.888102616308842
best k 1.0 6.52145259682649
best k 1.0 7.728045929992373
{1: (1.0, 6.888102616308842), 2: (1.0, 6.52145259682649), 3: (1.0, 7.728045929992373)}
best k 1.0 8.408430284308672
best k 0.5616435295000011 7.499882332471413
best k 0.23134208950000051 8.586682721678459
{1: (1.0, 8.408430284308672), 2: (0.5616435295000011, 7.499882332471413), 3: (0.23134208950000051, 8.586682721678459)}
best k 1.0 6.915151572870145
best k 1.0 6.45897208116136
best k 1.0 7.63492546812265
{1: (1.0, 6.915151572870145), 2: (1.0, 6.45897208116136), 3: (1.0, 7.63492546812265)}
best k 1.0 8.323678483317178
best k 0.594935817500001 7.391706636840973
best k 0.24091034550000048 8.447313934773915
{1: (1.0, 8.323678483317178), 2: (0.594935817500001, 7.391706636840973), 3: (0.24091034550000048, 8.447313934773915)}
best k 1.0 6.91225865446311
best k 1.0 6.64099377696999
best k 1.0 7.820652567100866
{1: (1.0, 6.91225865446311), 2: (1.0, 6.64099377696999), 3: (1.0, 7.820652567100866)}
best k 

In [26]:
def mean_cross_entropy_on_test_folds_joint_model(lambda_lm1, pair_piece_lms_self_orig, pair_piece_lms_other_orig,
                                                 cutoff_word_global_lm, folds):
    #print("lambda", lambda_lm1, "cutoff", cutoff_word_global_lm)
    pair_piece_lms_self = copy.deepcopy(pair_piece_lms_self_orig)
    pair_piece_lms_other = copy.deepcopy(pair_piece_lms_other_orig)
            
    cross_entropies = []  # get the cross-entropy for each fold
    for pair_num in folds.keys():
        test_set = folds[pair_num]
        s = 0
        count = 0
        for i, ref_info in enumerate(test_set):
            # assume we know the correct ref piece
            piece = ref_info[4]
            
            # get the weighted prob contribution from the self model
            self_prob = 1
            lm_self = pair_piece_lms_self[pair_num][piece]
            ref = ["<s>"] * (lm_self.order -1) + ref_info[0].split() + ["</s>"]
            
            for j in range(lm_self.order-1, len(ref)):
                target = ref[j]
                context = ref[j-(lm_self.order-1):j]
                ngram = context + [target]
                self_prob = self_prob * lm_self.prob_lidstone(ngram, lm_self.order)
                
            # get the weighted prob contribution from the other model
            other_prob = 1
            lm_other = pair_piece_lms_other[pair_num][piece]
            ref = ["<s>"] * (lm_other.order -1) + ref_info[0].split() + ["</s>"]
            for j in range(lm_other.order-1, len(ref)):
                target = ref[j]
                context = ref[j-(lm_other.order-1):j]
                ngram = context + [target]
                other_prob = other_prob * lm_other.prob_lidstone(ngram, lm_other.order)
            
            # combine the probs with correct lambda weights
            
            # calculate the correct lambda weights based on the cut-off word
            
            lambda_global = 1- lambda_lm1
            factor = 0 if cutoff_word_global_lm <=1 else (1-lambda_global)/(cutoff_word_global_lm-1)
            num_refs_f = pair_piece_lms_self[pair_num][piece].num_training_sents  # + 1 # to include the current one?
            current_lambda_lm1 = 1 - max([lambda_global + (factor * ((((cutoff_word_global_lm-1)-num_refs_f)))), lambda_global])

            
            prob = (current_lambda_lm1 * self_prob) + ((1-current_lambda_lm1) * other_prob)
            #final_ref_info = tuple(list(ref_info) + [prob, len(ref_info[0].split()), num_refs_f])
            #piece_references_and_probabilities_per_pair[pair_num].append(final_ref_info)
            
            s += log(prob)
            count +=1
            
            # update self language model, which starts empty for a piece
            # then will train as it encounters each piece
            #print(pair_piece_lms_self[pair_num][piece].num_training_sents, "train sents so far before")
            pair_piece_lms_self[pair_num][piece].train([ref_info[0].split()])

        cross_entropy = -s / count
        cross_entropies.append(cross_entropy)
    #print(lambda_lm1, cross_entropies, np.mean(cross_entropies))
    return np.mean(cross_entropies)

In [27]:
# NB MAY NOT USE THIS METHOD FOR OPTIMIZING THESE TWO
def get_best_params_cutoff_lambda_xval(n_self, n_other, k_self, k_other, pieces, folds,
                                       cutoff_range = [1,10],
                                       lambda_range=[0,1]):
    """An x-fold process run multiple times to find the best params of lambda (weighting of local language model)
    and cut-off word at which only the local model applies (with a gradual weighting)
    """
    results = {}  # store results for each model
    
    for test_cutoff in range(cutoff_range[0], cutoff_range[1]+1):
        pair_piece_lms_self = {}  # will produce all lms for each speaker for each piece with order test_cutoff
        pair_piece_lms_other = {}  # will produce all lms for each speaker for each piece with order test_cutoff
        for test_pair in folds.keys():
            # test pair is the list of refs to get the entropy results on in both cases
            # in 'other' model, referent_lm model is trained on refs on all other folds
            # in 'self' model, referent_lm model is trained on test_pair incrementally dynamically
                  #i.e. retrained with the ref added to the counts after that ref is tested
            pair_piece_lms_other[test_pair] = {}  # key piece, value language model for that piece
            pair_piece_lms_self[test_pair] = {}  # key piece, value language model for that piece
            for piece in pieces:
                for prob_model in ['other', 'self']:
                    if prob_model == 'other':
                        # train the model (smoothing can be changed as it doesn't affect training)

                        training_refs = []

                        for other_pair_num in folds.keys():
                            if other_pair_num == test_pair:
                                continue
                            training_refs.extend([ref for ref in filter(lambda x:x[4]==piece, folds[other_pair_num])])  
                        training_fold = [r[0] for r in training_refs]
                        #print("length of training fold for lm2", piece, test_pair, len(training_fold))
                        if len(training_fold) <1:
                            print("not enough data", piece, pair_num)
                            continue
                        pair_piece_lms_other[test_pair][piece] = mini_language_model(n_other, smoothing_k=k_other)    
                        pair_piece_lms_other[test_pair][piece].train([sent.split() for sent in training_fold])

                    elif prob_model == 'self':
                        # just initialize lm for piece, no need to train
                        pair_piece_lms_self[test_pair][piece] = mini_language_model(n_self, smoothing_k=k_self)    

        # run nelder mead to get best lambda
        best = optimize.minimize(
                mean_cross_entropy_on_test_folds_joint_model,
                lambda_range[0],    # first argument to be optimized (lambda)
                args=(pair_piece_lms_self, pair_piece_lms_other, test_cutoff, folds),  # other arguments to the function
                method='Nelder-Mead', # use nelder mead to find minima
                tol=0.0001, # to this degree of error
                options={'disp': False},
                bounds=[lambda_range]
        )
        best_lambda, best_ce = best.x[0], best.fun
        #print("best lambda", best_lambda, best_ce)
        results[test_cutoff] = (best_lambda, best_ce)
    
    print(results)
    cutoff = min(results.items(), key=lambda x:x[1][1])[0]
    lambda_lm1 = results[cutoff][0]
               
    return cutoff, lambda_lm1, results
    

In [371]:
def get_new_piece_lms_from_folds(n, k, pieces, folds, target_folds=None, train=False, heldout_extra_fold=None):
    """Returns a piece based language model objects of n=n, k=k, using the training folds"""
    #print('num folds', len(folds.keys()))
    pair_piece_lms_other = {}
    for test_pair in folds.keys():
        # test pair is the list of refs to get the entropy results on in both cases
        # in 'other' model, referent_lm model is trained on refs on all other folds
        # in 'self' model, referent_lm model is trained on test_pair incrementally dynamically
              #i.e. retrained with the ref added to the counts after that ref is tested
        if not test_pair in target_folds:
            continue
        pair_piece_lms_other[test_pair] = {}  # key piece, value language model for that piece
        count = 0
        for piece in pieces:
            # train the model (smoothing can be changed as it doesn't affect training)
            pair_piece_lms_other[test_pair][piece] = mini_language_model(n, smoothing_k=k)  
            if train:
                training_refs = []
                non_training_pairs = [test_pair]
                if not heldout_extra_fold is None:
                    # hold out either the heldout extra fold or it's the test pair, another one
                    random.seed(0)   #count)
                    count+=1
                    l = list(filter(lambda x:x!=test_pair,(folds.keys())))
                    shuffle(l)  # NB this may not have been shuffled before
                    heldout = l[0] if test_pair == heldout_extra_fold else heldout_extra_fold
                    #heldout = l[0]
                    non_training_pairs.append(heldout)
                num_training_pairs = 0
                for other_pair_num in folds.keys():
                    if other_pair_num in non_training_pairs:
                        continue
                    training_refs.extend([ref for ref in filter(lambda x:x[4]==piece, folds[other_pair_num])]) 
                    num_training_pairs+=1
                #print("num training pairs for lm", piece, test_pair, num_training_pairs)
                training_fold = [r[0] for r in training_refs]
                #print("length of training fold for lm", piece, test_pair, len(training_fold))
                if len(training_fold) <1:
                    print("not enough data", piece, test_pair)
                    continue
                pair_piece_lms_other[test_pair][piece].train([sent.split() for sent in training_fold])
    return pair_piece_lms_other

In [43]:
def generate_classifier_data_from_lms_and_params(pair_piece_lms_self_orig, pair_piece_lms_other_orig, pieces, lambda_lm1, cutoff_word_global_lm, folds, lexical=True):
    # Creates the appropriate language models for, and trains and tests classifiers for a given test pair
    # create the speaker-wise fold language models (using all other pairs except key)
   
    # Record all scores (probabilities) for each piece according to the language models in the folds 
    # (Which uses both other language models from observing other pairs and the local updating language model/pact model)
    piece_references_and_probabilities_per_pair = {}

    pair_piece_lms_self = copy.deepcopy(pair_piece_lms_self_orig)
    pair_piece_lms_other = copy.deepcopy(pair_piece_lms_other_orig)

  
    
    cross_entropies = []  # get the cross-entropy for each fold
    for pair_num in folds.keys():
        test_set = folds[pair_num]
        piece_references_and_probabilities_per_pair[pair_num] = []
        s = 0
        count = 0
        for i, ref_info in enumerate(test_set):
   
            target_piece = ref_info[4]
            
            piece_prob_dict = {} # gets the raw probs assigned to ref by all models
            
            for piece in pieces:
                # get the weighted prob contribution from the self model
                self_prob = 1
                lm_self = pair_piece_lms_self[pair_num][piece]
                ref = ["<s>"] * (lm_self.order -1) + ref_info[0].split() + ["</s>"]

                for j in range(lm_self.order-1, len(ref)):
                    target = ref[j]
                    context = ref[j-(lm_self.order-1):j]
                    ngram = context + [target]
                    self_prob = self_prob * lm_self.prob_lidstone(ngram, lm_self.order)

                # get the weighted prob contribution from the other model
                other_prob = 1
                lm_other = pair_piece_lms_other[pair_num][piece]
                ref = ["<s>"] * (lm_other.order -1) + ref_info[0].split() + ["</s>"]
                for j in range(lm_other.order-1, len(ref)):
                    target = ref[j]
                    context = ref[j-(lm_other.order-1):j]
                    ngram = context + [target]
                    other_prob = other_prob * lm_other.prob_lidstone(ngram, lm_other.order)

                # combine the probs with correct lambda weights

                # calculate the correct lambda weights based on the cut-off word
                lambda_global = 1- lambda_lm1
                factor = 0 if cutoff_word_global_lm <=1 else (1-lambda_global)/(cutoff_word_global_lm-1)
                num_refs_f = pair_piece_lms_self[pair_num][piece].num_training_sents  # + 1 # to include the current one?
                current_lambda_lm1 = 1 - max([lambda_global + (factor * ((((cutoff_word_global_lm-1)-num_refs_f)))), lambda_global])


                prob = (current_lambda_lm1 * self_prob) + ((1-current_lambda_lm1) * other_prob)
                
                
                    
                
                
                piece_prob_dict[piece] = (-log(prob)/len(ref_info[0].split()), num_refs_f)

                # only get the entropy for target piece
                if piece == target_piece:
                    s += log(prob)
                    count +=1
            
            
            
            # update self language model, which starts empty for a piece
            # then will train as it encounters each piece
            #print(pair_piece_lms_self[pair_num][piece].num_training_sents, "train sents so far before")
            pair_piece_lms_self[pair_num][target_piece].train([ref_info[0].split()])
            
        
            
            # could add moving averages for how this piece has been judged probability-wise by each piece model
            # built so far in this interaction by different models
            # for this instance, we don't know what the correct referent is, so we only have the probs from the models
            #  will be num shapes * num shapes
            # assumes the latest prob dict for this piece is a new instance for all shapes:
            init_probs_from_current = {k: [v[0]] for k, v in piece_prob_dict.items()}
            probs_assigned_to_pieces_so_far = {k:copy.deepcopy(init_probs_from_current) for k, v in piece_prob_dict.items()}
            # for previous instances, we do know what the correct referents were
            # so we can check what the effect of combining the current probs to those from previous positive examples
            # NB and negative ones too?
            # scroll forward from start of interaction up to but not including current ref
            for j in range(0, i):
                back_ref_info = piece_references_and_probabilities_per_pair[pair_num][j]
                back_ref_prob_dict = back_ref_info[5]
                back_ref_target_piece = back_ref_info[4]
                
                for k, v in back_ref_prob_dict.items():
                    if probs_assigned_to_pieces_so_far[back_ref_target_piece].get(k) is None:
                        probs_assigned_to_pieces_so_far[back_ref_target_piece][k] = []
                    probs_assigned_to_pieces_so_far[back_ref_target_piece][k].append(v[0])
            
            # will be num shapes * num shapes
            moving_average_prob_dicts_all_lms = {k:{} for k, v in piece_prob_dict.items()}
            for target_p in probs_assigned_to_pieces_so_far.keys():
                for a_p in probs_assigned_to_pieces_so_far[target_p].keys():
                    moving_average_prob_dicts_all_lms[target_p][a_p] = np.mean(probs_assigned_to_pieces_so_far[target_p][a_p])
                    
            
            
            final_ref_info = tuple(list(ref_info) + [piece_prob_dict, moving_average_prob_dicts_all_lms, len(ref_info[0].split())])
            piece_references_and_probabilities_per_pair[pair_num].append(final_ref_info)
            

        cross_entropy = -s / count
        cross_entropies.append(cross_entropy)
  
    return piece_references_and_probabilities_per_pair
    

In [30]:
def get_zscore_dict_for_prob_dict(prob_dict):
    probs  = [item[1] for item in sorted(prob_dict.items(), key=lambda x:x[0])] # alpha
    zscores = list(stats.zscore(np.array(probs)))
    def convert_nan(v):
        if np.isnan(v):
            return - 100
        return v
    return {k: convert_nan(v) for k, v in zip(sorted(prob_dict.keys()), zscores)}

get_zscore_dict_for_prob_dict({"K": 0.33, "Y": 0.35, "X": 0.33})

{'K': -0.7071067811865476, 'X': -0.7071067811865476, 'Y': 1.4142135623730951}

In [31]:
def generate_classifier_data_from_raw(folds, lexical=True, lm_features=True, moving_average=False):
    """Gets derived data from raw probs from language models"""
    final_data = []
    current_pair_num = None
    for current_pair_num in folds.keys():
        raw_data = folds[current_pair_num]
        for ref_info in raw_data:
            #('das orange l', 'r1_B', 5887, 204.521, 'L', prob_dict, ref_length)

            text, speaker, utt_id, end_time, target_piece, prob_dict, piece_lm_moving_average, ref_length = ref_info




            #if current_pair_piece_probs.get(piece) is None:
            #    current_pair_piece_probs[piece] = []


            feature_vector = {}

            if lexical:
                clean_utt = text.lower().split()
                word_dem = len(clean_utt)

                local_word_count = Counter()

                for word in clean_utt:
                    #word_counter[word] += (1/word_dem)   # for global
                    #word_piece_counter[piece+"__"+word] += (1/word_dem)  #for global
                    local_word_count[word] += 1

                for k, v in local_word_count.items():
                    feature_vector[k] = local_word_count[k]/word_dem



            if lm_features:
                length_weighted_prob_dict = {k:v[0] for k,v in prob_dict.items()}

                #for k, v in length_weighted_prob_dict.items():
                #    feature_vector['local_prob_' + k] = v

                zscore_dict = get_zscore_dict_for_prob_dict(length_weighted_prob_dict)

                for k, v in zscore_dict.items():
                    feature_vector['local_zscore_prob_' + k] = v 

                if moving_average:
                    #pass
                    # to get a smoother measure of the pacts so far (after the first one), 
                    # we calculate what the moving average lm score is for each piece lm
                    # with which previous pieces and current probs for this pair
                    # give for the current piece (not knowing what it is and assuming it's of each type)
                    # i.e. assuming it's of a shape (X), what's the moving average of all the piece lm's on this type of piece?
                    # you'd need num pieces * num pieces number of moving probs for it to be fair, not just for the target

                    for _target in piece_lm_moving_average.keys():
                        #if not _target == target_piece: # NB this is cheating!
                        #    continue
                        zscore_dict_average = get_zscore_dict_for_prob_dict(piece_lm_moving_average[_target])
                        for k, v in zscore_dict_average.items():  
                            if not k == _target:
                                continue
                            #feature_vector['local_prob_moving_zcore' + _target + ":" + k] = v
                            #feature_vector['local_prob_moving_zcore' + _target + ":" + k] =  piece_lm_moving_average[_target][k]
                            feature_vector['local_prob_moving_zcore' + _target + ":" + k] =  v

                #zscore_dict_average = get_zscore_dict_for_prob_dict(piece_lm_moving_average)

                #for k, v in zscore_dict_average.items():
                #    feature_vector['local_zscore_prob_moving_' + k] = v 


            #rank_dict = get_rank_dict_for_prob_dict(piece_lm_prob_local)

            #for k, v in rank_dict.items():
            #    feature_vector['local_prob_rank' + k] = v 
            final_data.append((feature_vector, target_piece))
    return final_data


In [32]:
# TRAINING AND VALIDATING OUR CLASSIFIER

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning, UndefinedMetricWarning
simplefilter("ignore", category=ConvergenceWarning)

def train_classifier(trainData):
    #print("Training Classifier...")
    return SklearnClassifier(LinearSVC(loss='squared_hinge')).train(trainData)
    #return SklearnClassifier(LogisticRegression()).train(trainData)

def predict_labels(samples, classifier):
    return classifier.classify_many(map(lambda t: t[0], samples))


In [33]:
if False:
    # Alternative method for finding best lambda local and cut-off - may not work as LMs of different size, so will always favour
    # high lambda for the local model which will always give higher probabilities
    # experiment with and optimize the best lambda weight on the local model and best cut-off point
    # in process get the cross validation results
    print(opt_params)
    n_self = opt_params["n_self"]
    n_other = opt_params["n_other"]
    k_self = opt_params["k_self"]
    k_other = opt_params["k_other"]
    cutoff_word_global_lm, lambda_local, x_val_results = get_best_params_cutoff_lambda_xval(n_self, n_other,
                                                                                            k_self, k_other,
                                                                                            good_pieces, training_folds,
                                                                                            cutoff_range=[1,5],
                                                                                            lambda_range=[0,1]) 
    opt_params["lambda_local"] = lambda_local
    opt_params["cutoff_word_global_lm"] = cutoff_word_global_lm

In [453]:
# finally get the test results with best language models (will have done this 7 times above in each xval)

def get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm, 
               lexical=True, 
               lm_features=True,
               moving_average=False,
               exhaustive_retrain=False,
               no_train=False,
               heldout_extra_fold=None):
    
    # First generate training data by getting the other/global probs from the other pair folds (k-fold style)
    # This is to simulate what it's like 
    # So the LM sizes are num_folds - 1
    # One problem is in the case where there is one pair to learn from, this means there is no 'other' model
    #, only one, so it has to use its own probabilities, which aren't representative
    training_folds = {k:references_per_pair[k] for k in filter(lambda x:x not in [test], references_per_pair.keys())}
    global_training_folds = {k:references_per_pair[k] for k in filter(lambda x:x not in [test], references_per_pair.keys())}
    extra_fold_penalty = 1 if heldout_extra_fold is not None else 0
    if (len(global_training_folds)-extra_fold_penalty) == 1:
        # this won't work at all as it needs to learn from other pairs, so just copy it to learn from its final model
        # not ideal but at least gives values
        global_training_folds['extra'] = global_training_folds[list(global_training_folds.keys())[0]]
        
    lms_global = get_new_piece_lms_from_folds(n_other, k_other, good_pieces,
                                              global_training_folds, target_folds=global_training_folds.keys(),
                                              train=True,
                                              heldout_extra_fold=heldout_extra_fold)
    #print("global model trained for training")
    lms_self = get_new_piece_lms_from_folds(n_self, k_self, good_pieces,
                                            training_folds,
                                            target_folds=training_folds.keys(),
                                            train=False)
    #print("local model trained for training")
    raw_train_data = generate_classifier_data_from_lms_and_params(lms_self,
                                                                  lms_global,
                                                                  good_pieces,
                                                                  lambda_local,
                                                                  cutoff_word_global_lm,
                                                                  training_folds)

    train_data = generate_classifier_data_from_raw(raw_train_data, lexical=lexical, lm_features=lm_features, moving_average=moving_average)
    #print("length training data", len(train_data))
    #print(train_data[0])


    # now generate the test data using LMs trained on all the other folds except heldout, to ensure the same sized LMs
    
    all_global_training_folds = {k:references_per_pair[k] for k in filter(lambda x:x not in [heldout], references_per_pair.keys())}
    extra_fold_penalty = 1 if heldout_extra_fold is not None else 0
    if (len(all_global_training_folds)-extra_fold_penalty) == 1:
        # this won't work at all as it needs to learn from other pairs, so just copy it to learn from its final model
        # not ideal but at least gives values
        all_global_training_folds['extra'] = global_training_folds['extra']
    
    test_folds ={k:references_per_pair[k] for k in filter(lambda x:x==test, references_per_pair.keys())}
    
    # if using all other folds, needs to take one other fold out for the LM training to ensure same size as training
    lms_global = get_new_piece_lms_from_folds(n_other, k_other, good_pieces,
                                                      all_global_training_folds, target_folds=[test], train=True,
                                                      heldout_extra_fold=heldout_extra_fold)
    #print("global model trained for testing")
    lms_self = get_new_piece_lms_from_folds(n_self, k_self, good_pieces,
                                                      test_folds, target_folds=[test], train=False)

    #print("local model trained for testing")
    raw_test_data = generate_classifier_data_from_lms_and_params(lms_self,
                                                                 lms_global,
                                                                 good_pieces,
                                                                 lambda_local,
                                                                 cutoff_word_global_lm,
                                                                 test_folds)


    test_data = generate_classifier_data_from_raw(raw_test_data, lexical=lexical, lm_features=lm_features, moving_average=moving_average)
    #print(test_data[0])

    if no_train:
        train_data = []  # just start with no data
    
    prf = None
    if exhaustive_retrain:
        y_true = [x[1] for x in test_data]
        y_pred = []
        # do exhaustive incremental testing on current exmaple then retraining given last example
        for t in range(0, len(test_data)):
            if len(set([x[1] for x in train_data])) < 2:  # only one class seen so far, can't train yet
                random.seed(t)  # guess a piece using index t as random seed
                shuffle(good_pieces)
                guess = good_pieces[0]
                y_pred.extend(guess)
            else:
                # predict and train model with current data
                cl = train_classifier(train_data)
                y_pred.extend(predict_labels([test_data[t]], cl))
                
            feats, label = test_data[t]
            feats = {k:v for k, v in feats.items()}
            train_data.append((feats, label))  # add current example just attempted to training data
            
    else:
        cl = train_classifier(train_data)
        #print(train_data[0])
        y_true = [x[1] for x in test_data]
        y_pred = predict_labels(test_data, cl)
    prf = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
    #print(prf)
    #print(classification_report(y_true, y_pred))
    return prf[2], y_pred, y_true, accuracy

In [449]:
# Do optimization to find best cut-off and lambda local param values (rather than just cross-entropy as not straightforward)
# find best lambda local and cut-off word through cross-val testing, pick one with highest accuracy
OPTIMIZING = False
if OPTIMIZING:
    n_other = opt_params['n_other']
    n_self = opt_params['n_self']
    k_other = opt_params['k_other']
    k_self = opt_params['k_self']

    print(TEST, HELDOUT)
    cross_val_data = {k:references_per_pair[k] for k in filter(lambda x:x !=TEST, references_per_pair.keys())}
    results = {}
    results_file = open(datetime.datetime.now().isoformat() + ".csv", "w")
    results_file.write(",".join(['n_other', 'n_self', 'cutoff_word_global_lm', 'k_other', 'k_self', 'lambda_local', 'fscore']) + "\n")
    
    best_f = 0
    best_params = []
    for n_other in range(1,3): #2
        for n_self in range(1,3): #4
            for k_other_raw in range(5,105,5): #80
                k_other = k_other_raw/100
                for k_self_raw in range(5,105,5): #1_600
                    k_self = k_self_raw/100
                    for cutoff_word_global_lm in range(1,11): #16_000
                        # assume convex - not quite right but nearly 
                        prev = 0
                        for lambda_local_raw in range(0,105,5): #320_000 params max, at least 32_000!
                            lambda_local = lambda_local_raw/100
                            all_scores = []
                            for test in cross_val_data.keys():
                                heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
                                #print("heldout", heldout) 
                                #print(cross_val_data.keys(), test)
                                fscore, preds, labels, accuracy = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                                                            )
                                all_scores.append(fscore)
                            mean_f = np.mean(all_scores)
                            test_params = [n_other, n_self, cutoff_word_global_lm, k_other, k_self, lambda_local]
                            print(test_params, mean_f)
                            if mean_f < prev: # assume convex - not quite right but nearly 
                                break
                            prev = mean_f
                            results[",".join([str(f) for f in test_params])] = mean_f
                            if mean_f == best_f:
                                best_params = [best_params, test_params]
                            elif mean_f > best_f:
                                best_f = mean_f
                                best_params = test_params
                                print("best", best_f, best_params)
                    #results_file = open("results" + datetime.time)
                    #for row in sorted(results.items(), key=lambda x:x[1], reverse=True):
    
    for params, result in sorted(results.items(), key=lambda x:x[1], reverse=True):
        results_file.write(params + "," + str(result) + "\n")
    results_file.close()


In [450]:
# if already optimized look at results
#df = pd.read_csv(open("2023-06-12T00:56:28.534472.csv"))
#df = pd.read_csv(open("2023-06-14T01:39:17.361190.csv"))
#df = pd.read_csv(open("2023-06-14T11:31:32.265307.csv"))
#df = pd.read_csv(open("2023-06-14T12:34:31.314703.csv"))
#df = pd.read_csv(open("2023-06-14T12:46:30.442480.csv"))
#df = pd.read_csv(open("2023-06-14T15:48:07.727393.csv"))
df = pd.read_csv(open("2023-06-14T17:17:24.306257.csv"))
opt_params['lambda_local'] = df.head(1).lambda_local[0]
opt_params["cutoff_word_global_lm"] = df.head(1).cutoff_word_global_lm[0]

In [451]:
df.head(20)

,n_other,n_self,cutoff_word_global_lm,k_other,k_self,lambda_local,fscore
0,2,2,1,0.610468,1.0,0.05,0.860194
1,2,2,2,0.610468,1.0,0.05,0.860067
2,2,2,4,0.610468,1.0,0.05,0.859863
3,2,2,6,0.610468,1.0,0.05,0.857367
4,2,2,3,0.610468,1.0,0.10,0.856486
5,2,2,5,0.610468,1.0,0.05,0.856402
6,2,2,10,0.610468,1.0,0.10,0.856126
7,2,2,9,0.610468,1.0,0.10,0.856072
8,2,2,3,0.610468,1.0,0.05,0.855252
9,2,2,7,0.610468,1.0,0.10,0.854881


In [38]:
# from best so far 2023-06-16T011.59 0.8635429156885808 [1, 1, 6, 0.9, 0.1, 0.1]
# [n_other, n_self, cutoff_word_global_lm, k_other, k_self, lambda_local]
opt_params['n_other'] = 1
opt_params['n_self'] = 1 
opt_params["cutoff_word_global_lm"] = 6
opt_params['k_other'] = 0.9
opt_params['k_self'] = 0.1 
opt_params['lambda_local'] = 0.1

In [39]:
xval_results = {}

In [47]:
# 1) re-run best cross-val result
# get extra heldout fold to ensure we're still using 5 folds for training
cross_val_data = {k:references_per_pair[k] for k in filter(lambda x:x !=TEST, references_per_pair.keys())}

print(opt_params)
n_other = opt_params['n_other']
n_self = opt_params['n_self']
k_other = opt_params['k_other']
k_self = opt_params['k_self']
lambda_local = opt_params['lambda_local']
cutoff_word_global_lm = opt_params["cutoff_word_global_lm"]

cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, False]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

{'n_self': 1, 'k_self': 0.1, 'n_other': 1, 'k_other': 0.9, 'lambda_local': 0.1, 'cutoff_word_global_lm': 6}
[0.8871311102690413, 0.8087874978790278, 0.8926487915385203, 0.8899784067072645, 0.8968750266827189, 0.8886062889291302, 0.7946812595512271] 0.8655297687938474


In [48]:
# 2) Get the baseline of lexical info only, no updating/retraining or lms
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lm_features = False)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, False, False]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.6976847374847374, 0.8058614566470366, 0.895668919444112, 0.8931468445574603, 0.8536309338434843, 0.8564809646531358, 0.7659589533530414] 0.8240618299975725


In [49]:
# 3) Get lexical + global model only lm (should be similar to lexical)
lambda_local = 0
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, False]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.7246986463828569, 0.7918846376934613, 0.9145174415089884, 0.8970742250169593, 0.8537042900288949, 0.8541041652473775, 0.7689553382268967] 0.8292769634436334


In [50]:
# 4) Get lexical + local model only lm
lambda_local = 1
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, False]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.7686290376290377, 0.8124080183029923, 0.8966329779681271, 0.9026170744132981, 0.8630223835993067, 0.864958116053062, 0.7805252333446817] 0.8412561201872152


In [51]:
# 5) Try without lexical (nb lambda local and cutoff not optimized for this)
lambda_local = opt_params["lambda_local"]
cutoff_word_global_lm = opt_params["cutoff_word_global_lm"]
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lexical=False)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, False, True, False]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.8697043740573152, 0.7736168639971956, 0.8949503313171588, 0.8533104694213736, 0.8780294846581191, 0.8870000220112586, 0.7645720702581644] 0.8458833736743695


In [52]:
# 6) Try local only without lexical (nb lambda local and cutoff not optimized for this)
lambda_local = 1
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lexical=False)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, False, True, False]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.675582867192135, 0.5751349772634603, 0.7985183953199408, 0.7332651425675122, 0.7939372465491673, 0.7661477456648974, 0.6387100894925264] 0.71161378057852


In [53]:
# 7) Try global only without lexical (nb lambda local and cutoff not optimized for this)
lambda_local = 0
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lexical=False)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, False, True, False]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.6884963388864317, 0.7657131697408088, 0.8808514308284283, 0.8749988052211283, 0.8579328549862049, 0.853103021391952, 0.7373586156511767] 0.808350605243733


In [54]:
# 8) see what difference exahustive retraining makes, firstly lexical only, no lm features
lambda_local = opt_params["lambda_local"]
cutoff_word_global_lm = opt_params["cutoff_word_global_lm"]
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lm_features=False, exhaustive_retrain=True)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, False, True]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.8685030408188303, 0.829141396743478, 0.9110090265963389, 0.9003430391290285, 0.8764684023331268, 0.8924742188931777, 0.7802727854354273] 0.8654588442784866


In [55]:
# 9) see what difference exahustive retraining makes, lexical + lm features 
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                       exhaustive_retrain=True)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, True]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.8866883858108371, 0.8034349932654528, 0.9213174449096011, 0.8992745773683755, 0.8968750266827189, 0.8931788966826546, 0.7591878765786372] 0.8657081716140397


In [56]:
# 10) retraining makes, lexical + lm features (local only) 
lambda_local = 1
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in cross_val_data.keys():
    heldout = EXTRA_HELDOUT if test == HELDOUT else HELDOUT  # only not the normal heldout when that's being tested on
    #print("heldout", heldout) 
    f, preds, y = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                       exhaustive_retrain=True)
    print(test, f)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, True]
xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

r1 0.8741857475857476
r2 0.8097413928258987
r3 0.9027462923071812
r4 0.8929444691178358
r5 0.8750562532286699
r7 0.8923263268216141
r8 0.7735965794314416
[0.8741857475857476, 0.8097413928258987, 0.9027462923071812, 0.8929444691178358, 0.8750562532286699, 0.8923263268216141, 0.7735965794314416] 0.8600852944740556


In [58]:
comparison_results = {}
for sys1 in xval_results.keys():
    for sys2 in xval_results.keys():
        if sys1 == sys2:
            continue
        if sys1 + " V " + sys2 in comparison_results.keys() or sys2 + " V " + sys1 in comparison_results.keys():
            continue
        cl1_preds = xval_results[sys1][2]
        cl2_preds = xval_results[sys2][2]
        sig = calculate_mcnemar_test(cl1_preds, cl2_preds, all_y, alpha=0.05, exact=True)
        comparison_results[sys1 + " V " + sys2] = [xval_results[sys1][1], xval_results[sys2][1]] + list(sig)
        

print(len(comparison_results))
count = 1
for k, v in comparison_results.items():
    print(count, ")", k, v)
    count+=1

45
1 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,False,False [0.8655297687938474, 0.8240618299975725, 22.0, 2.8793907495367242e-06, True, True, True]
2 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,0,True,True,False [0.8655297687938474, 0.8292769634436334, 24.0, 7.395947454563746e-05, True, True, True]
3 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,True,True,False [0.8655297687938474, 0.8412561201872152, 22.0, 0.0035448128470210523, True, True, False]
4 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,False,True,False [0.8655297687938474, 0.8458833736743695, 18.0, 0.00013083876154198454, True, True, True]
5 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,False,True,False [0.8655297687938474, 0.71161378057852, 53.0, 3.76128685784491e-41, True, True, True]
6 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,0,False,True,False [0.8655297687938474, 0.808350605243733, 20.0, 1.1680094670975612e-12, True, True, True]
7 ) 1,1,6,0.9,0.1,0.1,True,Tr

In [ ]:
# Final system (in the middle of optimizing r6, r3 16/06/9am )
1 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,10,0.75,0.1,0.15,True,False,False [0.8635429156885808, 0.8240618299975725, 22.0, 4.348703111758687e-06, True, True, True]
2 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,1,0.75,0.1,0,True,True,False [0.8635429156885808, 0.8302171684556623, 22.0, 7.010573979919962e-05, True, True, True]
3 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,1,0.75,0.1,1,True,True,False [0.8635429156885808, 0.8412561201872152, 23.0, 0.005582667351023373, True, True, False]
4 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,10,0.75,0.1,0.15,False,True,False [0.8635429156885808, 0.8445599814154114, 23.0, 8.715478657320524e-05, True, True, True]
5 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,1,0.75,0.1,1,False,True,False [0.8635429156885808, 0.7115210426781088, 55.0, 2.274449585488219e-40, True, True, True]
6 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,1,0.75,0.1,0,False,True,False [0.8635429156885808, 0.8101090393857525, 23.0, 1.4884906475803411e-10, True, True, True]
7 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,10,0.75,0.1,0.15,True,False,True [0.8635429156885808, 0.8654588442784866, 25.0, 0.8898839652765371, False, False, False]
8 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,10,0.75,0.1,0.15,True,True,True [0.8635429156885808, 0.8670818039494403, 16.0, 0.735878800856881, False, False, False]
9 ) 1,1,10,0.75,0.1,0.15,True,True,False V 1,1,1,0.75,0.1,1,True,True,True [0.8635429156885808, 0.8600852944740556, 27.0, 0.4426260049464714, False, False, False]
Beats all except the re-training competitors 7,8.9 which it is the same as

In [ ]:
# Final system (in the middle of optimizing r6, r3, r7, 20/06/5pm )
1 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,False,False [0.8655297687938474, 0.8240618299975725, 22.0, 2.8793907495367242e-06, True, True, True]
2 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,0,True,True,False [0.8655297687938474, 0.8292769634436334, 24.0, 7.395947454563746e-05, True, True, True]
3 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,True,True,False [0.8655297687938474, 0.8412561201872152, 22.0, 0.0035448128470210523, True, True, False]
4 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,False,True,False [0.8655297687938474, 0.8458833736743695, 18.0, 0.00013083876154198454, True, True, True]
5 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,False,True,False [0.8655297687938474, 0.71161378057852, 53.0, 3.76128685784491e-41, True, True, True]
6 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,0,False,True,False [0.8655297687938474, 0.808350605243733, 20.0, 1.1680094670975612e-12, True, True, True]
7 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,False,True [0.8655297687938474, 0.8654588442784866, 25.0, 1.0, False, False, False]
8 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,True,True [0.8655297687938474, 0.8657081716140397, 17.0, 0.9999999999999998, False, False, False]
9 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,True,True,True [0.8655297687938474, 0.8600852944740556, 25.0, 0.358143301806138, False, False, False]
Beats all except the re-training competitors 7,8.9 which it is the same as

In [ ]:
# For final test, re-estimate LM params as on a bigger sample
# TODO for now don't re-optimise lambda local and cutoff
if False: # perhaps not, try to use the same size LMs as in the xval by holding one out


    n_range = [1,2,3]
    k_val_range = [0.00000001, 1]

    l = list(filter(lambda x:x!=TEST, ['r1', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8']))
    all_n_self = []
    all_k_self = []
    all_n_other = []
    all_k_other = []

    training_folds = {k:references_per_pair[k] for k in filter(lambda x:x not in [TEST], references_per_pair.keys())}
    n_self, k_self, lm_opt_results_self = get_params_for_lowest_entropy_on_fold_test(training_folds, good_pieces,
                                                                  n_range, k_val_range, prob_model='self')


    all_n_self.append(n_self)
    all_k_self.append(k_self)

    training_folds = {k:references_per_pair[k] for k in filter(lambda x:x not in [TEST], references_per_pair.keys())}
    n_other, k_other, lm_opt_results_other = get_params_for_lowest_entropy_on_fold_test(training_folds, good_pieces,
                                                                  n_range, k_val_range, prob_model='other')

    all_n_other.append(n_other)
    all_k_other.append(k_other)

    opt_params["n_self"] = round(np.mean(all_n_self))  # NB python rounds down 0.5 but shouldn't matter
    opt_params["k_self"] = np.mean(all_k_self)

    opt_params["n_other"] = round(np.mean(all_n_other)) # NB python rounds down 0.5 but shouldn't matter
    opt_params["k_other"] = np.mean(all_k_other)
    print(opt_params)

In [59]:
# Final test results (on R2, possibly removing random as heldout for the final result)
#opt_params['lambda_local'] = 0.2
#opt_params['cutoff_word_global_lm'] = 4
test_results = {}
#random.seed(0)
#l = list(filter(lambda x:x!=test, ['r1', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8']))
#shuffle(l)
#heldout = l[0]
#print(heldout)


In [60]:
# 1) best system from cross val
# Note we are introducing a HELDOUT_EXTRA fold, to ensure while all the k folds (7) training data is used,
# the size of the LMs used to produce those probs is the same as in the 7-fold xval on that data (i.e. 5 (not 6)) folds

# determine what best to leave out for extra heldout for global model (leave out one close to the mean 86/median??)
# overall mean = 0.8655297687938474:
#[r1 0.8871311102690413 (5), r2 0.8087874978790278 (6), 
#r3 0.8926487915385203 (2), r4 0.8899784067072645 (3), 
#r5 0.8968750266827189 (1), r7 0.8886062889291302 (4), 
#r8 0.7946812595512271 (7)]

# crf global +lexical mean = 0.8292769634436334 (could be indicative of different global models):
#[r1 0.7246986463828569 (7) ,  r2 0.7918846376934613 (5), 
# r3 0.9145174415089884 (1),  r4 0.8970742250169593 (2), 
# r5 0.8537042900288949 (4),  r7 0.8541041652473775 (3), 
# r8 0.7689553382268967 (6)] 

# crf global without lexical mean = 0.8101090393857525 (could be indicative of different global models):
#[r1 0.7035747702589807 (7) , r2 0.7657131697408088 (5) , 
#r3 0.8969597351106516 (1) , r4 0.8823844381067611 (2),
#r5 0.8266118187023894 (4) , r7 0.853103021391952 (3),
# r8 0.7424163223887241 (6) ]

#seed 0 for random alternative
#r1 0.8459473042740722
#r2 0.841488456372273
#r4 0.8302973126293945
#r5 0.8493825773532347
#r7 0.8805434338542419
#r8 0.8390350268176637

#r1 100 refs
#r2 170 refs
#r3 265 refs
#r4 285 refs
#r5 143 refs
#r6 258 refs
#r7 352 refs
#r8 326 refs

print(TEST, HELDOUT, EXTRA_HELDOUT)

print(opt_params)
n_other = opt_params['n_other']
n_self = opt_params['n_self']
k_other = opt_params['k_other']
k_self = opt_params['k_self']
lambda_local = opt_params['lambda_local']
cutoff_word_global_lm = opt_params["cutoff_word_global_lm"]

cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    #print("heldout", heldout)
    f, preds, y = get_fscore_and_preds(references_per_pair, test, HELDOUT, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, False]
print(exp_params)
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

r6 r3 r7
{'n_self': 1, 'k_self': 0.1, 'n_other': 1, 'k_other': 0.9, 'lambda_local': 0.1, 'cutoff_word_global_lm': 6}
[1, 1, 6, 0.9, 0.1, 0.1, True, True, False]
[0.8805434338542419] 0.8805434338542419


In [61]:
# 2) Get the baseline of lexical info only, no updating/retraining or lms
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lm_features = False)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, False, False]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.834707644846363] 0.834707644846363


In [62]:
# 3) Get lexical + global model only lm (should be similar to lexical)
lambda_local = 0
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, False]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.8375122990795726] 0.8375122990795726


In [63]:
# 4) Get lexical + local model only lm
lambda_local = 1
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, False]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.8492712094602781] 0.8492712094602781


In [64]:
# 5) Try without lexical (nb lambda local and cutoff not optimized for this)
lambda_local = opt_params["lambda_local"]
cutoff_word_global_lm = opt_params["cutoff_word_global_lm"]
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lexical=False, heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, False, True, False]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.8879653298798018] 0.8879653298798018


In [65]:
# 6) Try local only without lexical (nb lambda local and cutoff not optimized for this)
lambda_local = 1
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lexical=False, heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, False, True, False]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.7488221315217394] 0.7488221315217394


In [66]:
# 7) Try global only without lexical (nb lambda local and cutoff not optimized for this)
lambda_local = 0
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lexical=False,heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, False, True, False]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.8383477473836394] 0.8383477473836394


In [67]:
# 8) see what difference exahustive retraining makes, firstly lexical only, no lm features
lambda_local = opt_params["lambda_local"]
cutoff_word_global_lm = opt_params["cutoff_word_global_lm"]
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                      lm_features=False, exhaustive_retrain=True, heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, False, True]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.9006807283839668] 0.9006807283839668


In [68]:
# 9) see what difference exahustive retraining makes, lexical + lm features 
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                       exhaustive_retrain=True, heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, True]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.8916486628252694] 0.8916486628252694


In [69]:
# 10) exahustive retraining, lexical + lm features  (local only)
lambda_local = 1
cutoff_word_global_lm = 1
cross_val_results = []
all_preds = []
all_y = []
for test in [TEST]:
    f, preds, y = get_fscore_and_preds(references_per_pair, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                       exhaustive_retrain=True, heldout_extra_fold=EXTRA_HELDOUT)
    all_preds.extend(preds)
    all_y.extend(y)
    cross_val_results.append(f)
exp_params = [n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, True, True, True]
test_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
print(cross_val_results, np.mean(cross_val_results))

[0.9012381797222825] 0.9012381797222825


In [70]:
comparison_results_test = {}
for sys1 in test_results.keys():
    for sys2 in test_results.keys():
        if sys1 == sys2:
            continue
        if sys1 + " V " + sys2 in comparison_results_test.keys() or sys2 + " V " + sys1 in comparison_results_test.keys():
            continue
        cl1_preds = test_results[sys1][2]
        cl2_preds = test_results[sys2][2]
        sig = calculate_mcnemar_test(cl1_preds, cl2_preds, all_y, alpha=0.05, exact=True)
        comparison_results_test[sys1 + " V " + sys2] = [test_results[sys1][1], test_results[sys2][1]] + list(sig)
        

statistic=0.000, p-value=0.000
Different proportions of errors (reject H0)
statistic=0.000, p-value=0.001
Different proportions of errors (reject H0)
statistic=3.000, p-value=0.035
Different proportions of errors (reject H0)
statistic=2.000, p-value=0.688
Same proportions of errors (fail to reject H0)
statistic=5.000, p-value=0.000
Different proportions of errors (reject H0)
statistic=6.000, p-value=0.052
Same proportions of errors (fail to reject H0)
statistic=0.000, p-value=0.062
Same proportions of errors (fail to reject H0)
statistic=0.000, p-value=0.250
Same proportions of errors (fail to reject H0)
statistic=1.000, p-value=0.125
Same proportions of errors (fail to reject H0)
statistic=4.000, p-value=1.000
Same proportions of errors (fail to reject H0)
statistic=1.000, p-value=0.375
Same proportions of errors (fail to reject H0)
statistic=2.000, p-value=0.001
Different proportions of errors (reject H0)
statistic=13.000, p-value=0.001
Different proportions of errors (reject H0)
sta

In [71]:
count = 1
for k, v in comparison_results_test.items():
    print(count, ")", k, v)
    count+=1

1 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,False,False [0.8805434338542419, 0.834707644846363, 0.0, 0.00048828125, True, True, True]
2 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,0,True,True,False [0.8805434338542419, 0.8375122990795726, 0.0, 0.0009765625, True, True, True]
3 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,True,True,False [0.8805434338542419, 0.8492712094602781, 3.0, 0.03515625, True, False, False]
4 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,False,True,False [0.8805434338542419, 0.8879653298798018, 2.0, 0.6875, False, False, False]
5 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,False,True,False [0.8805434338542419, 0.7488221315217394, 5.0, 7.878384167270269e-08, True, True, True]
6 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,0,False,True,False [0.8805434338542419, 0.8383477473836394, 6.0, 0.052478790283203125, False, False, False]
7 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,False,True [

In [ ]:
with r6 test, r3 heldout, r7 extra (interim June 20th 5pm)
1 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,False,False [0.8805434338542419, 0.834707644846363, 0.0, 0.00048828125, True, True, True]
2 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,0,True,True,False [0.8805434338542419, 0.8375122990795726, 0.0, 0.0009765625, True, True, True]
3 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,True,True,False [0.8805434338542419, 0.8492712094602781, 3.0, 0.03515625, True, False, False]
4 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,False,True,False [0.8805434338542419, 0.8879653298798018, 2.0, 0.6875, False, False, False]
5 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,False,True,False [0.8805434338542419, 0.7488221315217394, 5.0, 7.878384167270269e-08, True, True, True]
6 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,0,False,True,False [0.8805434338542419, 0.8383477473836394, 6.0, 0.052478790283203125, False, False, False]
7 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,False,True [0.8805434338542419, 0.9006807283839668, 0.0, 0.0625, False, False, False]
8 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,6,0.9,0.1,0.1,True,True,True [0.8805434338542419, 0.8916486628252694, 0.0, 0.25, False, False, False]
9 ) 1,1,6,0.9,0.1,0.1,True,True,False V 1,1,1,0.9,0.1,1,True,True,True [0.8805434338542419, 0.9012381797222825, 1.0, 0.125, False, False, False]
Beats baseline 1) lexical no retrain :) 2) GLOBAL only 3) LOCAL only  5) local only no lexical
Doesn't beat/same as:  4) no lexical  6) global no lexical, 7) exhaustive retrain lexical only  8) exhaustive retrain  9) exhaustive retrain local only


# Experiment 2: Removing the training data by one pair each time, down to 1 pair

In [76]:
# for a given setup of reduced training folds, get the right global params for a given number of reduced
# the self model should have the same params
opt_params_reduced = {}
opt_params_reduced['n_self'] = opt_params['n_self']
opt_params_reduced['k_self'] = opt_params['k_self']

n_range = [1,2,3]
k_val_range = [0.00000001, 1]

for num_training_pairs in range(2,7):
    print(num_training_pairs)
    training_folds = list(filter(lambda x:x not in [TEST], ['r1', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8']))
    random.seed(0)
    shuffle(training_folds)
    training_folds = training_folds[:num_training_pairs]
    print(training_folds)
    reduced_data = {k:references_per_pair[k] for k in filter(lambda x:x in training_folds, references_per_pair.keys())}
    n_other, k_other, lm_opt_results_other = get_params_for_lowest_entropy_on_fold_test(reduced_data, good_pieces,
                                                              n_range, k_val_range, prob_model='other')
    opt_params_reduced["n_other_{0}".format(num_training_pairs-1)] = n_other
    opt_params_reduced["k_other_{0}".format(num_training_pairs-1)] = k_other




2
['r7', 'r4']
best k 1.0 8.459471990135716
best k 1.0 8.763609062649696
best k 1.0 10.055934938495543
{1: (1.0, 8.459471990135716), 2: (1.0, 8.763609062649696), 3: (1.0, 10.055934938495543)}
3
['r7', 'r4', 'r3']
best k 1.0 8.026707033041179
best k 1.0 7.432905649117651
best k 0.6107955295000012 8.838865075654837
{1: (1.0, 8.026707033041179), 2: (1.0, 7.432905649117651), 3: (0.6107955295000012, 8.838865075654837)}
4
['r7', 'r4', 'r3', 'r1']
best k 1.0 8.089230134874791
best k 0.8771338335000014 7.357918206807465
best k 0.35310797750000056 8.526929044996598
{1: (1.0, 8.089230134874791), 2: (0.8771338335000014, 7.357918206807465), 3: (0.35310797750000056, 8.526929044996598)}
5
['r7', 'r4', 'r3', 'r1', 'r8']
best k 1.0 8.433653723253368
best k 0.7868252255000012 7.695774551527665
best k 0.3282042975000006 8.8602205720436
{1: (1.0, 8.433653723253368), 2: (0.7868252255000012, 7.695774551527665), 3: (0.3282042975000006, 8.8602205720436)}
6
['r7', 'r4', 'r3', 'r1', 'r8', 'r5']
best k 1.0 8.32

In [77]:
print(opt_params_reduced)

{'n_self': 1, 'k_self': 0.1, 'n_other_1': 1, 'k_other_1': 1.0, 'n_other_2': 2, 'k_other_2': 1.0, 'n_other_3': 2, 'k_other_3': 0.8771338335000014, 'n_other_4': 2, 'k_other_4': 0.7868252255000012, 'n_other_5': 2, 'k_other_5': 0.594935817500001}


In [78]:
# Do optimization to find best cut-off and lambda local param values (rather than just cross-entropy as not straightforward)
# find best lambda local and cut-off word through cross-val testing, pick one with highest accuracy
OPTIMIZING = False
if OPTIMIZING:
    n_self = opt_params_reduced['n_self']  
    k_self = opt_params_reduced['k_self']

    results = {}

    for num_training_pairs in range(1,6):
        print(num_training_pairs)  
        n_other = opt_params_reduced["n_other_{0}".format(num_training_pairs)]
        k_other = opt_params_reduced["k_other_{0}".format(num_training_pairs)] 
        
        best_f = 0
        best_params = []
        for cutoff_word_global_lm in range(1,11):
            for lambda_local_raw in range(0,105,5):
                lambda_local = lambda_local_raw/100
                all_scores = []
                for test in ['r1', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8']:
                    training_folds = list(filter(lambda x:x!=test, ['r1', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8']))
                    random.seed(0)
                    shuffle(training_folds)
                    training_folds = training_folds[:num_training_pairs] + [test]
                    print(training_folds)
                    reduced_data = {k:references_per_pair[k] for k in filter(lambda x:x in training_folds, references_per_pair.keys())}
                    fscore, preds, labels = get_fscore_and_preds(reduced_data, test, "", n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm)
                    all_scores.append(fscore)
                mean_f = np.mean(all_scores)
                test_params = [num_training_pairs, n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local]
                print(test_params, mean_f)
                results[",".join([str(f) for f in test_params])] = mean_f
                if mean_f == best_f:
                    best_params = [best_params, test_params]
                elif mean_f > best_f:
                    best_f = mean_f
                    best_params = test_params
                    print("best", best_f, best_params)
    #results_file = open("results" + datetime.time)
    #for row in sorted(results.items(), key=lambda x:x[1], reverse=True):
    results_file = open(datetime.datetime.now().isoformat() + "_REDUCED.csv", "w")
    results_file.write(",".join(['num_training_pairs', 'n_other', 'n_self', 'cutoff_word_global_lm', 'k_other', 'k_self', 'lambda_local', 'fscore']) + "\n")
    for params, result in sorted(results.items(), key=lambda x:x[1], reverse=True):
        results_file.write(params + "," + str(result) + "\n")
    results_file.close()

In [158]:
results_file = open("2023-06-21T08:17:12.999981_reduced.csv", "a")

In [156]:
print(num_training_pairs)

2


In [159]:
# Do optimization to find best cut-off and lambda local param values (rather than just cross-entropy as not straightforward)
# find best lambda local and cut-off word through cross-val testing, pick one with highest accuracy
OPTIMIZING = False
if OPTIMIZING:
    
    # assume these are the same (could be incorrect)
    n_self = opt_params['n_self'] 
    k_self = opt_params['k_self']

    print(TEST, HELDOUT, EXTRA_HELDOUT)
    
    results = {}
    results_file = open(datetime.datetime.now().isoformat() + "_reduced.csv", "w")
    results_file.write(",".join(['num_pairs', 'n_other', 'n_self', 'cutoff_word_global_lm', 'k_other', 'k_self', 'lambda_local', 'fscore']) + "\n")
    
    for num_pairs in range(1,6):  # Full model is using 6 folds for training in cross val
        
        #cross_val_pairs = list(filter(lambda x:x not in [TEST, HELDOUT, EXTRA_HELDOUT], references_per_pair.keys()))[:num_pairs-1]
        #cross_val_pairs.extend([HELDOUT, EXTRA_HELDOUT])
        #print(num_pairs, cross_val_pairs)
        #cross_val_data = {k:references_per_pair[k] for k in filter(lambda x:x in cross_val_pairs, references_per_pair.keys())}
        
        
        best_f = 0
        best_params = []
        for n_other in range(1,2): #1
            for k_other_raw in range(5,105,5): #20
                k_other = k_other_raw/100
                for cutoff_word_global_lm in range(1,11): #200
                    # assume convex - not quite right but nearly 
                    prev = 0
                    for lambda_local_raw in range(0,105,5): #4_000 params max, at least 4_00!
                        lambda_local = lambda_local_raw/100
                        all_scores = []
                        for test in list(filter(lambda x:x!=TEST, references_per_pair.keys())):
                            heldout = HELDOUT if test!=HELDOUT else EXTRA_HELDOUT # use the heldout when training on full dataset to simulate same lm sizes as in development
                            other_pairs = list(filter(lambda x:x not in [heldout, test], 
                                                      [HELDOUT, EXTRA_HELDOUT] + list(filter(lambda x:x not in [HELDOUT, EXTRA_HELDOUT], ['r1', 'r2', 'r3', 'r4', 'r5', 'r7', 'r8']))))
                            cross_val_pairs = [heldout, test] + other_pairs[:num_pairs-1]
                            #print(cross_val_pairs, test, heldout)
                            cross_val_data = {k:references_per_pair[k] for k in filter(lambda x:x in cross_val_pairs, references_per_pair.keys())}
                            #print("heldout", heldout) 
                            #print(cross_val_data.keys(), test)
                            fscore, preds, labels = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                                                        )
                            all_scores.append(fscore)
                        mean_f = np.mean(all_scores)
                        test_params = [num_pairs, n_other, n_self, cutoff_word_global_lm, k_other, k_self, lambda_local]
                        print(test_params, mean_f)
                        if mean_f < prev: # assume convex - not quite right but nearly 
                            break
                        prev = mean_f
                        results[",".join([str(f) for f in test_params])] = mean_f
                        if mean_f == best_f:
                            best_params = [best_params, test_params]
                        elif mean_f > best_f:
                            best_f = mean_f
                            best_params = test_params
                            print("best", best_f, best_params)
                #results_file = open("results" + datetime.time)
                #for row in sorted(results.items(), key=lambda x:x[1], reverse=True):
    
    for params, result in sorted(results.items(), key=lambda x:x[1], reverse=True):
        results_file.write(params + "," + str(result) + "\n")
    results_file.close()


r6 r3 r7


/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[3, 1, 1, 1, 0.05, 0.1, 0.0] 0.7076018365090361
best 0.7076018365090361 [3, 1, 1, 1, 0.05, 0.1, 0.0]
[3, 1, 1, 1, 0.05, 0.1, 0.05] 0.8147189559540168
best 0.8147189559540168 [3, 1, 1, 1, 0.05, 0.1, 0.05]
[3, 1, 1, 1, 0.05, 0.1, 0.1] 0.8101582275370649


/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[3, 1, 1, 2, 0.05, 0.1, 0.0] 0.7076018365090361
[3, 1, 1, 2, 0.05, 0.1, 0.05] 0.8144358113273691
[3, 1, 1, 2, 0.05, 0.1, 0.1] 0.813478214266298


...

[5, 1, 1, 7, 0.1, 0.1, 0.15] 0.8360662829017738
[5, 1, 1, 8, 0.1, 0.1, 0.0] 0.7866418826343883
[5, 1, 1, 8, 0.1, 0.1, 0.05] 0.8347056444413427
[5, 1, 1, 8, 0.1, 0.1, 0.1] 0.8346916719734824
[5, 1, 1, 9, 0.1, 0.1, 0.0] 0.7866418826343883
[5, 1, 1, 9, 0.1, 0.1, 0.05] 0.8348058832808983
[5, 1, 1, 9, 0.1, 0.1, 0.1] 0.8349463019641359
[5, 1, 1, 9, 0.1, 0.1, 0.15] 0.8357155469404536
[5, 1, 1, 9, 0.1, 0.1, 0.2] 0.8383982637337589
[5, 1, 1, 9, 0.1, 0.1, 0.25] 0.8379399364436617
[5, 1, 1, 10, 0.1, 0.1, 0.0] 0.7866418826343883
[5, 1, 1, 10, 0.1, 0.1, 0.05] 0.8346044660006079
[5, 1, 1, 10, 0.1, 0.1, 0.1] 0.8349946896029243
[5, 1, 1, 10, 0.1, 0.1, 0.15] 0.8335854553152097
[5, 1, 1, 1, 0.15, 0.1, 0.0] 0.7886553884456985
[5, 1, 1, 1, 0.15, 0.1, 0.05] 0.8315050122784238
[5, 1, 1, 1, 0.15, 0.1, 0.1] 0.8278340387333645
[5, 1, 1, 2, 0.15, 0.1, 0.0] 0.7886553884456985
[5, 1, 1, 2, 0.15, 0.1, 0.05] 0.831048493896292
[5, 1, 1, 2, 0.15, 0.1, 0.1] 0.8284264540965245
[5, 1, 1, 3, 0.15, 0.1, 0.0] 0.78865538844

[5, 1, 1, 4, 0.35, 0.1, 0.0] 0.7886274487169398
[5, 1, 1, 4, 0.35, 0.1, 0.05] 0.8382375573795956
[5, 1, 1, 4, 0.35, 0.1, 0.1] 0.8357848490122015
[5, 1, 1, 5, 0.35, 0.1, 0.0] 0.7886274487169398
[5, 1, 1, 5, 0.35, 0.1, 0.05] 0.8374438769034
[5, 1, 1, 5, 0.35, 0.1, 0.1] 0.8394175473933798
[5, 1, 1, 5, 0.35, 0.1, 0.15] 0.8377668601222137
[5, 1, 1, 6, 0.35, 0.1, 0.0] 0.7886274487169398
[5, 1, 1, 6, 0.35, 0.1, 0.05] 0.8378406426286598
[5, 1, 1, 6, 0.35, 0.1, 0.1] 0.8402359291298025
[5, 1, 1, 6, 0.35, 0.1, 0.15] 0.8398988589507954
[5, 1, 1, 7, 0.35, 0.1, 0.0] 0.7886274487169398
[5, 1, 1, 7, 0.35, 0.1, 0.05] 0.8370602000680184
[5, 1, 1, 7, 0.35, 0.1, 0.1] 0.8402975710376281
[5, 1, 1, 7, 0.35, 0.1, 0.15] 0.8410996538824976
[5, 1, 1, 7, 0.35, 0.1, 0.2] 0.839739778742378
[5, 1, 1, 8, 0.35, 0.1, 0.0] 0.7886274487169398
[5, 1, 1, 8, 0.35, 0.1, 0.05] 0.836110555477255
[5, 1, 1, 8, 0.35, 0.1, 0.1] 0.8405432971322672
[5, 1, 1, 8, 0.35, 0.1, 0.15] 0.8409982746121855
[5, 1, 1, 8, 0.35, 0.1, 0.2] 0.83959

[5, 1, 1, 3, 0.55, 0.1, 0.1] 0.8359394561569305
[5, 1, 1, 4, 0.55, 0.1, 0.0] 0.7900793586327072
[5, 1, 1, 4, 0.55, 0.1, 0.05] 0.8377565692621368
[5, 1, 1, 4, 0.55, 0.1, 0.1] 0.8367547141253066
[5, 1, 1, 5, 0.55, 0.1, 0.0] 0.7900793586327072
[5, 1, 1, 5, 0.55, 0.1, 0.05] 0.8385439600968123
[5, 1, 1, 5, 0.55, 0.1, 0.1] 0.8393887330834836
[5, 1, 1, 5, 0.55, 0.1, 0.15] 0.8383640878738028
[5, 1, 1, 6, 0.55, 0.1, 0.0] 0.7900793586327072
[5, 1, 1, 6, 0.55, 0.1, 0.05] 0.8371053429694798
[5, 1, 1, 6, 0.55, 0.1, 0.1] 0.839847732192388
[5, 1, 1, 6, 0.55, 0.1, 0.15] 0.8379412287336617
[5, 1, 1, 7, 0.55, 0.1, 0.0] 0.7900793586327072
[5, 1, 1, 7, 0.55, 0.1, 0.05] 0.8354273036539416
[5, 1, 1, 7, 0.55, 0.1, 0.1] 0.8386885827260399
[5, 1, 1, 7, 0.55, 0.1, 0.15] 0.838366782316755
[5, 1, 1, 8, 0.55, 0.1, 0.0] 0.7900793586327072
[5, 1, 1, 8, 0.55, 0.1, 0.05] 0.8349048892954318
[5, 1, 1, 8, 0.55, 0.1, 0.1] 0.8373984092656196
[5, 1, 1, 8, 0.55, 0.1, 0.15] 0.8387288555395739
[5, 1, 1, 8, 0.55, 0.1, 0.2] 0.83

[5, 1, 1, 1, 0.75, 0.1, 0.0] 0.7899554773335098
[5, 1, 1, 1, 0.75, 0.1, 0.05] 0.8351167077802563
[5, 1, 1, 1, 0.75, 0.1, 0.1] 0.8330864309758353
[5, 1, 1, 2, 0.75, 0.1, 0.0] 0.7899554773335098
[5, 1, 1, 2, 0.75, 0.1, 0.05] 0.834530656484797
[5, 1, 1, 2, 0.75, 0.1, 0.1] 0.8343038393758697
[5, 1, 1, 3, 0.75, 0.1, 0.0] 0.7899554773335098
[5, 1, 1, 3, 0.75, 0.1, 0.05] 0.8375688377321516
[5, 1, 1, 3, 0.75, 0.1, 0.1] 0.835388566991749
[5, 1, 1, 4, 0.75, 0.1, 0.0] 0.7899554773335098
[5, 1, 1, 4, 0.75, 0.1, 0.05] 0.8398741505664281
[5, 1, 1, 4, 0.75, 0.1, 0.1] 0.8375873095461454
[5, 1, 1, 5, 0.75, 0.1, 0.0] 0.7899554773335098
[5, 1, 1, 5, 0.75, 0.1, 0.05] 0.8381208492045678
[5, 1, 1, 5, 0.75, 0.1, 0.1] 0.8390073674453072
[5, 1, 1, 5, 0.75, 0.1, 0.15] 0.8382957042519708
[5, 1, 1, 6, 0.75, 0.1, 0.0] 0.7899554773335098
[5, 1, 1, 6, 0.75, 0.1, 0.05] 0.8375765317391366
[5, 1, 1, 6, 0.75, 0.1, 0.1] 0.8385261427254885
[5, 1, 1, 6, 0.75, 0.1, 0.15] 0.8390026066634275
[5, 1, 1, 6, 0.75, 0.1, 0.2] 0.841

[5, 1, 1, 8, 0.95, 0.1, 0.05] 0.8389370793435964
[5, 1, 1, 8, 0.95, 0.1, 0.1] 0.8395904243230332
[5, 1, 1, 8, 0.95, 0.1, 0.15] 0.8378494975964433
[5, 1, 1, 9, 0.95, 0.1, 0.0] 0.7889614115568602
[5, 1, 1, 9, 0.95, 0.1, 0.05] 0.8382504178529129
[5, 1, 1, 9, 0.95, 0.1, 0.1] 0.8375833147754903
[5, 1, 1, 10, 0.95, 0.1, 0.0] 0.7889614115568602
[5, 1, 1, 10, 0.95, 0.1, 0.05] 0.8367375063652994
[5, 1, 1, 10, 0.95, 0.1, 0.1] 0.8371977189663273
[5, 1, 1, 10, 0.95, 0.1, 0.15] 0.8377349715752332
[5, 1, 1, 10, 0.95, 0.1, 0.2] 0.83873614294577
[5, 1, 1, 10, 0.95, 0.1, 0.25] 0.8382519625459546
[5, 1, 1, 1, 1.0, 0.1, 0.0] 0.7938358842685733
[5, 1, 1, 1, 1.0, 0.1, 0.05] 0.8360359984201625
[5, 1, 1, 1, 1.0, 0.1, 0.1] 0.8301412117098739
[5, 1, 1, 2, 1.0, 0.1, 0.0] 0.7938358842685733
[5, 1, 1, 2, 1.0, 0.1, 0.05] 0.8363811771596793
[5, 1, 1, 2, 1.0, 0.1, 0.1] 0.8299933842696577
[5, 1, 1, 3, 1.0, 0.1, 0.0] 0.7938358842685733
[5, 1, 1, 3, 1.0, 0.1, 0.05] 0.8388493339784118
[5, 1, 1, 3, 1.0, 0.1, 0.1] 0.83677

In [461]:
# if already optimized look at results
df = pd.read_csv(open("2023-06-21T08:17:12.999981_reduced.csv"))
local_df = df[df['num_pairs']==1]
local_df

,num_pairs,n_other,n_self,cutoff_word_global_lm,k_other,k_self,lambda_local,fscore
0,1,1,1,7,0.10,0.1,0.10,0.783648
1,1,1,1,8,0.20,0.1,0.25,0.783520
2,1,1,1,8,0.10,0.1,0.10,0.782771
3,1,1,1,6,0.10,0.1,0.10,0.781791
4,1,1,1,4,0.15,0.1,0.05,0.781598
...,...,...,...,...,...,...,...,...
754,1,1,1,6,1.00,0.1,0.00,0.603963
755,1,1,1,7,1.00,0.1,0.00,0.603963
756,1,1,1,8,1.00,0.1,0.00,0.603963
757,1,1,1,9,1.00,0.1,0.00,0.603963


In [171]:
for num_training_pairs in range(1,6):
    local_df = df[df['num_pairs']==num_training_pairs]
    opt_params_reduced['n_self_{}'.format(num_training_pairs)] = int(local_df.iloc[0].n_self)
    opt_params_reduced['k_self_{}'.format(num_training_pairs)] = local_df.iloc[0].k_self
    opt_params_reduced['n_other_{}'.format(num_training_pairs)] = int(local_df.iloc[0].n_other)
    opt_params_reduced['k_other_{}'.format(num_training_pairs)] = local_df.iloc[0].k_other
    opt_params_reduced['lambda_local_{}'.format(num_training_pairs)] = local_df.iloc[0].lambda_local
    opt_params_reduced['cutoff_word_global_lm_{}'.format(num_training_pairs)] = int(local_df.iloc[0].cutoff_word_global_lm)

In [172]:
# Assume the cutoff and lambda for 6 pairs is the same as the global, as trained on all
opt_params_reduced['n_self_{}'.format(6)] = opt_params['n_self']
opt_params_reduced['k_self_{}'.format(6)] = opt_params['k_self']
opt_params_reduced['n_other_{}'.format(6)] = opt_params['n_other']
opt_params_reduced['k_other_{}'.format(6)] = opt_params['k_other']
opt_params_reduced['lambda_local_{}'.format(6)] = opt_params['lambda_local']
opt_params_reduced['cutoff_word_global_lm_{}'.format(6)] = opt_params['cutoff_word_global_lm']

In [173]:
opt_params_reduced

{'n_self': 1,
 'k_self': 0.1,
 'n_other_1': 1,
 'k_other_1': 0.1,
 'n_other_2': 1,
 'k_other_2': 1.0,
 'n_other_3': 1,
 'k_other_3': 0.95,
 'n_other_4': 1,
 'k_other_4': 0.75,
 'n_other_5': 1,
 'k_other_5': 0.7,
 'lambda_local_1': 0.1,
 'cutoff_word_global_lm_1': 7,
 'lambda_local_2': 0.8,
 'cutoff_word_global_lm_2': 6,
 'lambda_local_3': 0.65,
 'cutoff_word_global_lm_3': 8,
 'lambda_local_4': 0.4,
 'cutoff_word_global_lm_4': 5,
 'lambda_local_5': 0.3,
 'cutoff_word_global_lm_5': 7,
 'lambda_local_6': 0.1,
 'cutoff_word_global_lm_6': 6,
 'n_self_1': 1,
 'k_self_1': 0.1,
 'n_self_2': 1,
 'k_self_2': 0.1,
 'n_self_3': 1,
 'k_self_3': 0.1,
 'n_self_4': 1,
 'k_self_4': 0.1,
 'n_self_5': 1,
 'k_self_5': 0.1,
 'n_self_6': 1,
 'k_self_6': 0.1,
 'n_other_6': 1,
 'k_other_6': 0.9}

In [441]:
reduced_xval_results = {}

In [446]:
# 1) re-run best cross-val result
print(TEST, HELDOUT, EXTRA_HELDOUT)

local_results = {}
for num_pairs in range(1,7):
    print("num pairs", num_pairs)
    #print(opt_params_reduced)

    n_self = opt_params_reduced['n_self_{}'.format(min([6,num_pairs]))]
    k_self = opt_params_reduced['k_self_{}'.format(min([6,num_pairs]))]
    n_other = opt_params_reduced['n_other_{}'.format(min([6,num_pairs]))]
    k_other = opt_params_reduced['k_other_{}'.format(min([6,num_pairs]))]
    

    lambda_local = opt_params_reduced['lambda_local_{}'.format(min([6,num_pairs]))]
    cutoff_word_global_lm = opt_params_reduced['cutoff_word_global_lm_{}'.format(min([6,num_pairs]))]

    lexical = True
    lm_features = False
    retrain = True

    exp_params = [num_pairs, n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, lexical, lm_features, retrain]
    print(exp_params)
    cross_val_results = []
    all_preds = []
    all_y = []
    for i, test in enumerate(['r1', 'r2', 'r3', 'r4', 'r5', 'r7', 'r8']):
        #random.seed(0)  # note in training this was always 0
        #shuffle(training_folds)
        
        heldout = HELDOUT if test!=HELDOUT else EXTRA_HELDOUT # use the heldout when training on full dataset to simulate same lm sizes as in development
        other_pairs = list(filter(lambda x:x not in [heldout, test], 
                                  [HELDOUT, EXTRA_HELDOUT] + list(filter(lambda x:x not in [HELDOUT, EXTRA_HELDOUT], ['r1', 'r2', 'r3', 'r4', 'r5', 'r7', 'r8']))))
        cross_val_pairs = [heldout, test] + other_pairs[:num_pairs-1]
        print(cross_val_pairs, test, heldout)
        cross_val_data = {k:references_per_pair[k] for k in filter(lambda x:x in cross_val_pairs, references_per_pair.keys())}
        #print("heldout", heldout) 
        #print(cross_val_data.keys(), test)
        fscore, preds, labels = get_fscore_and_preds(cross_val_data, test, heldout, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                                      lexical=lexical, lm_features=lm_features, exhaustive_retrain=retrain,
                                                      no_train=False)
                                                    

        all_preds.extend(preds)
        all_y.extend(labels)
        cross_val_results.append(fscore)

    reduced_xval_results[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
    print(cross_val_results, np.mean(cross_val_results))
    local_results[num_pairs] = np.mean(cross_val_results)
print(local_results)

r6 r3 r7
num pairs 1
[1, 1, 1, 7, 0.1, 0.1, 0.1, True, False, True]
['r3', 'r1'] r1 r3
['r3', 'r2'] r2 r3
['r7', 'r3'] r3 r7
['r3', 'r4'] r4 r3
['r3', 'r5'] r5 r3
['r3', 'r7'] r7 r3
['r3', 'r8'] r8 r3
[0.8365459861775651, 0.7595053753707299, 0.8818035272373219, 0.8545454547272463, 0.8274919673251356, 0.8365571040552329, 0.7303249059815995] 0.8181106172678331
num pairs 2
[2, 1, 1, 6, 1.0, 0.1, 0.8, True, False, True]
['r3', 'r1', 'r7'] r1 r3
['r3', 'r2', 'r7'] r2 r3
['r7', 'r3', 'r1'] r3 r7
['r3', 'r4', 'r7'] r4 r3
['r3', 'r5', 'r7'] r5 r3
['r3', 'r7', 'r1'] r7 r3
['r3', 'r8', 'r7'] r8 r3
[0.8832258969788381, 0.7644119388237036, 0.8922453382562306, 0.8932206398648354, 0.8515268219843817, 0.867458561771847, 0.7225028823337328] 0.8392274400019384
num pairs 3
[3, 1, 1, 8, 0.95, 0.1, 0.65, True, False, True]
['r3', 'r1', 'r7', 'r2'] r1 r3
['r3', 'r2', 'r7', 'r1'] r2 r3
['r7', 'r3', 'r1', 'r2'] r3 r7
['r3', 'r4', 'r7', 'r1'] r4 r3
['r3', 'r5', 'r7', 'r1'] r5 r3
['r3', 'r7', 'r1', 'r2'] r7 r3

In [447]:
for num_pairs in range(1,7):
    print("num pairs", num_pairs)
    #print(opt_params_reduced)
    reduced_comparison_results = {}
    for sys1 in reduced_xval_results.keys():
        if not int(sys1.split(",")[0]) == num_pairs:
            continue
        for sys2 in reduced_xval_results.keys():
            if not int(sys2.split(",")[0]) == num_pairs:
                continue
            if sys1 == sys2:
                continue
            if sys1 + " V " + sys2 in reduced_comparison_results.keys() or sys2 + " V " + sys1 in reduced_comparison_results.keys():
                continue
            cl1_preds = reduced_xval_results[sys1][2]
            cl2_preds = reduced_xval_results[sys2][2]
            sig = calculate_mcnemar_test(cl1_preds, cl2_preds, all_y, alpha=0.05, exact=True)
            reduced_comparison_results[sys1 + " V " + sys2] = [round(reduced_xval_results[sys1][1],3), round(reduced_xval_results[sys2][1],3)] + list(sig)


    count = 1
    for k, v in reduced_comparison_results.items():
        print(count, ")", k, v)
        count+=1
    print("*" * 30)


num pairs 1
statistic=72.000, p-value=0.000
Different proportions of errors (reject H0)
statistic=54.000, p-value=0.001
Different proportions of errors (reject H0)
statistic=21.000, p-value=0.000
Different proportions of errors (reject H0)
1 ) 1,1,1,7,0.1,0.1,0.1,True,True,False V 1,1,1,7,0.1,0.1,0.1,True,False,False [0.7836480487128183, 0.6452247609627264, 72.0, 1.114096399640308e-28, True, True, True]
2 ) 1,1,1,7,0.1,0.1,0.1,True,True,False V 1,1,1,7,0.1,0.1,0.1,True,False,True [0.7836480487128183, 0.8181106172678331, 54.0, 0.0005846387710440814, True, True, True]
3 ) 1,1,1,7,0.1,0.1,0.1,True,False,False V 1,1,1,7,0.1,0.1,0.1,True,False,True [0.6452247609627264, 0.8181106172678331, 21.0, 6.1738493352342854e-55, True, True, True]
******************************
num pairs 2
statistic=53.000, p-value=0.000
Different proportions of errors (reject H0)
statistic=39.000, p-value=0.008
Different proportions of errors (reject H0)
statistic=19.000, p-value=0.000
Different proportions of errors 

In [ ]:
# now get the test results

In [372]:
reduced_results_test = {}

In [407]:
local_results = {}

for num_pairs in range(1,8):
    print("num pairs", num_pairs)
    #print(opt_params_reduced)

    n_self = opt_params_reduced['n_self_{}'.format(min([6,num_pairs]))]
    k_self = opt_params_reduced['k_self_{}'.format(min([6,num_pairs]))]
    n_other = opt_params_reduced['n_other_{}'.format(min([6,num_pairs]))]
    k_other = opt_params_reduced['k_other_{}'.format(min([6,num_pairs]))]


    lambda_local = opt_params_reduced['lambda_local_{}'.format(min([6,num_pairs]))]
    cutoff_word_global_lm = opt_params_reduced['cutoff_word_global_lm_{}'.format(min([6,num_pairs]))]
    
    
    #if num_pairs in [4,5,6]:
    #    n_other = opt_params_reduced['n_other_{}'.format(6)]
    #    k_other = opt_params_reduced['k_other_{}'.format(6)]
     #   lambda_local = opt_params_reduced['lambda_local_{}'.format(6)]
     #   cutoff_word_global_lm = opt_params_reduced['cutoff_word_global_lm_{}'.format(6)]

    lexical = True
    lm_features = True
    retrain = False

    exp_params = [num_pairs, n_other, n_self, cutoff_word_global_lm, k_other,k_self, lambda_local, lexical, lm_features, retrain]
    print(exp_params)
    cross_val_results = []
    all_preds = []
    all_y = []
    for i, test in enumerate([TEST]):
        #random.seed(0)  # note in training this was always 0
        #shuffle(training_folds)
        
        other_pairs = list(filter(lambda x:x not in [HELDOUT, TEST, EXTRA_HELDOUT], ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8']))
        #random.seed(num_pairs)
        #shuffle(other_pairs)
        cross_val_pairs = [TEST, HELDOUT, EXTRA_HELDOUT] + other_pairs
        cross_val_pairs = cross_val_pairs[:num_pairs+1]
        extra_heldout = EXTRA_HELDOUT  if num_pairs > 6 else None # HELDOUT
        print(cross_val_pairs, test, HELDOUT, extra_heldout)
        cross_val_data = {k:references_per_pair[k] for k in filter(lambda x:x in cross_val_pairs, references_per_pair.keys())}
        #print("heldout", heldout) 
        #print(cross_val_data.keys(), test)
        
        fscore, preds, labels = get_fscore_and_preds(cross_val_data, test, HELDOUT, n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm,
                                                     heldout_extra_fold=extra_heldout,
                                                      lexical=lexical, lm_features=lm_features, exhaustive_retrain=retrain,
                                                      no_train=False)
                                                    

        all_preds.extend(preds)
        all_y.extend(labels)
        cross_val_results.append(fscore)

    reduced_results_test[','.join([str(p) for p in exp_params])] = (cross_val_results, np.mean(cross_val_results),all_preds)
    print(cross_val_results, np.mean(cross_val_results))
    local_results[num_pairs] = np.mean(cross_val_results)
print(local_results)

num pairs 1
[1, 1, 1, 7, 0.1, 0.1, 0.1, True, False, True]
['r6', 'r3'] r6 r3 None
[0.8626845322568283] 0.8626845322568283
num pairs 2
[2, 1, 1, 6, 1.0, 0.1, 0.8, True, False, True]
['r6', 'r3', 'r7'] r6 r3 None
[0.8862258349153512] 0.8862258349153512
num pairs 3
[3, 1, 1, 8, 0.95, 0.1, 0.65, True, False, True]
['r6', 'r3', 'r7', 'r1'] r6 r3 None
[0.8895084671613147] 0.8895084671613147
num pairs 4
[4, 1, 1, 5, 0.75, 0.1, 0.4, True, False, True]
['r6', 'r3', 'r7', 'r1', 'r2'] r6 r3 None
[0.9000728246516821] 0.9000728246516821
num pairs 5
[5, 1, 1, 7, 0.7, 0.1, 0.3, True, False, True]
['r6', 'r3', 'r7', 'r1', 'r2', 'r4'] r6 r3 None
[0.9042132031544717] 0.9042132031544717
num pairs 6
[6, 1, 1, 6, 0.9, 0.1, 0.1, True, False, True]
['r6', 'r3', 'r7', 'r1', 'r2', 'r4', 'r5'] r6 r3 None
[0.9042132031544717] 0.9042132031544717
num pairs 7
[7, 1, 1, 6, 0.9, 0.1, 0.1, True, False, True]
['r6', 'r3', 'r7', 'r1', 'r2', 'r4', 'r5', 'r8'] r6 r3 r7
[0.9006807283839668] 0.9006807283839668
{1: 0.862684

In [462]:
for num_training_pairs in range(1,8):
    print("num training pairs", num_training_pairs)
    reduced_comparison_results_test = {}
    for sys1 in reduced_results_test.keys():
        #print(sys1)
        if not int(sys1.split(",")[0]) == num_training_pairs:
            continue
        #print("getting 1")
        for sys2 in reduced_results_test.keys():
            if not int(sys2.split(",")[0]) == num_training_pairs:
                continue
            #print("getting 2")
            if sys1 == sys2:
                continue
            #print("getting 3")
            if sys1 + " V " + sys2 in reduced_comparison_results_test.keys() or sys2 + " V " + sys1 in reduced_comparison_results_test.keys():
                continue
            #print("getting 4")
            cl1_preds = reduced_results_test[sys1][2]
            cl2_preds = reduced_results_test[sys2][2]
            sig = calculate_mcnemar_test(cl1_preds, cl2_preds, all_y, alpha=0.05, exact=True)
            reduced_comparison_results_test[sys1 + " V " + sys2] = [round(reduced_results_test[sys1][1],3), round(reduced_results_test[sys2][1],3)] + list(sig)

    count = 1
    for k, v in reduced_comparison_results_test.items():
        print(count, ")", k, v)
        count+=1
    print("*" * 30)

num training pairs 1
statistic=5.000, p-value=0.581
Same proportions of errors (fail to reject H0)
statistic=3.000, p-value=1.000
Same proportions of errors (fail to reject H0)
statistic=5.000, p-value=0.581
Same proportions of errors (fail to reject H0)
1 ) 1,1,1,7,0.1,0.1,0.1,True,True,False V 1,1,1,7,0.1,0.1,0.1,True,False,False [0.818, 0.632, 5.0, 0.5810546875, False, False, False]
2 ) 1,1,1,7,0.1,0.1,0.1,True,True,False V 1,1,1,7,0.1,0.1,0.1,True,False,True [0.818, 0.863, 3.0, 1.0, False, False, False]
3 ) 1,1,1,7,0.1,0.1,0.1,True,False,False V 1,1,1,7,0.1,0.1,0.1,True,False,True [0.632, 0.863, 5.0, 0.5810546875, False, False, False]
******************************
num training pairs 2
statistic=2.000, p-value=0.453
Same proportions of errors (fail to reject H0)
statistic=1.000, p-value=0.625
Same proportions of errors (fail to reject H0)
statistic=3.000, p-value=1.000
Same proportions of errors (fail to reject H0)
1 ) 2,1,1,6,1.0,0.1,0.8,True,True,False V 2,1,1,6,1.0,0.1,0.8,True,

In [377]:
df = pd.read_csv(open("2023-06-12T00:56:28.534472.csv"))
opt_params['lambda_local'] = df.head(1).lambda_local[0]
opt_params["cutoff_word_global_lm"] = df.head(1).cutoff_word_global_lm[0]

In [ ]:
df

In [ ]:
# test results
print(opt_params)
n_other = opt_params['n_other']
n_self = opt_params['n_self']
k_other = opt_params['k_other']
k_self = opt_params['k_self']
lambda_local = opt_params['lambda_local']
cutoff_word_global_lm = opt_params["cutoff_word_global_lm"]
fscore = get_fscore(references_per_pair, TEST, "r3", n_other, n_self, k_other, k_self, lambda_local, cutoff_word_global_lm)

In [ ]:
# Note for lexical features only to work, they only do so after training on all 7 folds, at 6 severe degradation
# lm features work well early on after 2-3 examples

In [ ]:
def plot_piece_lm_values_for_pair_piece(data, piece, moving_average=False):
    piece_plots = {p:[] for p in data[0][5].keys()}
    for ref_info in data:
        text, speaker, utt_id, end_time, target_piece, prob_dict, piece_lm_moving_average, ref_length = ref_info
        
        if not target_piece == piece:  # only focus on target piece
            continue
        print(text, speaker, end_time)
        if moving_average is True:
            # how does each piece lm give a probability to THIS target piece over time
            for lm_piece, inner_prob_dict in piece_lm_moving_average.items():
                for inner_piece_lm, prob in piece_lm_moving_average[lm_piece].items():
                    if not inner_piece_lm == piece:
                        continue
                    piece_plots[lm_piece].append(prob)
        else:
            # if just raw probs
            for lm_piece, prob in prob_dict.items():
                piece_plots[lm_piece].append(prob[0])
    
    #for k, v in piece_plots.items():
    #    print(k,v)
                
    #‘b’	blue
    #‘g’	green
    #‘r’	red  (255, 0, 0 )
    #‘c’	cyan
    #‘m’	magenta
    #‘y’	yellow
    #‘k’	black
    #‘w’	white

    #orange (255, 128, 0)
    #grey (160, 160, 160)
    #pink (255, 153, 204)
    #brown (153, 76, 0)
    #purple (102, 0, 204)

    colour_map = {'F': (160/255, 160/255, 160/255), 'I': "blue" , 'L': (255/255, 128/255, 0/255),
                  'N': (102/255, 0/255, 204/255),
                  'P': (255/255, 153/255, 204/255), 'T': "green", 'U': "yellow", 'V': "blue",
                  'W' :"green", 'X': "red", 'Y': (153/255, 76/255, 0/255), 'Z': "blue"}

    for lm_piece in sorted(piece_plots.keys()):
        probs = piece_plots[lm_piece]
        target_text = "(TARGET)" if lm_piece == piece else ""
        linestyle = "solid" if lm_piece == piece else "dashed"
        marker = "o" if lm_piece == piece else "none"
        plt.plot(probs, label=lm_piece + " " + target_text, color=colour_map[lm_piece],
                linestyle=linestyle, marker=marker)


    # plot lines
    #plt.plot(moving_average([m[0] for m in method1]), label = "active: mixed models with k=0.015 for previous")
    #plt.plot(moving_average([m[0] for m in method2]), label = "interactive: retraining language model with incrementing data")
    #plt.plot(moving_average([m[0] for m in method3]), label = "static baseline")
    #plt.plot(moving_average([m[0] for m in method4]), label = "interactive: mixed models with k=0.015 for previous + recency")
    #plt.plot(moving_average([m[0] for m in method5]), label = "active: retraining language model with incrementing data + recency")
    #plt.plot(moving_average([m[0] for m in method6]), label = "interactive only + recency")
    #plt.plot(moving_average([m[0] for m in method7]), label = "interactive only")
    plt.legend(bbox_to_anchor=(0.1, 1))
    plt.show()


In [ ]:
for piece in sorted(good_pieces):
    plot_piece_lm_values_for_pair_piece(raw_test_data['r2'], piece, moving_average=True)

In [ ]:
das lange blaue r2_B 206.009
das lange blaue r2_B 221.956
der eins r2_A 223.186
ganz geraden r2_B 660.089
das ding r2_B 669.892
was r2_B 669.892
es">wie's r2_B 673.518
das ganz ganz lange r2_B 1022.337
das blaue r2_B 1023.311
den langen balken r2_A 1188.479
den langen balken r2_A 1307.589
das lange blaue r2_A 1524.858
der balken r2_A 1630.552
des balkens r2_A 1645.637



unser z r8_B 343.829
das andere z r8_B 365.865
das andere z r8_B 550.645
das andere z r8_B 554.113
dieses z r8_B 570.425
das z r8_B 644.144
dieses r8_A 646.046
das z r8_B 862.151
dieses z r8_B 1041.278
dieses z r8_B 1288.31
das blaue z r8_A 1524.868
das z r8_A 1689.259
das z r8_A 1937.962
das blaue z r8_A 1940.632
das z r8_A 2171.232
das z r8_A 2193.904
das z r8_A 2283.683
das z r8_A 2506.665
das blaue z r8_A 2844.14
das blaue z r8_A 2846.798
dieses r8_A 2851.145
dem blauen z r8_A 2864.463

In [ ]:
# competitor 1: global language model only
global_only_params = {k:v for k, v in opt_params.items()}
global_only_params['lambda_local'] = 0 # no use of local lm

train_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, global_only_params, lexical=True)
test_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, global_only_params, lexical=True)

cl = trainClassifier(train_data)
print(train_data[0])
y_true = [x[1] for x in test_data]
y_pred_global_only = predictLabels(test_data, cl)
prf = precision_recall_fscore_support(y_true, y_pred_global_only, average='weighted')
print(prf)
print(classification_report(y_true, y_pred_global_only))

In [ ]:
# competitor 2: local language model only
local_only_params = {k:v for k, v in opt_params.items()}
local_only_params['lambda_local'] = 1 # only uses local, no global

train_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, local_only_params, lexical=True)
test_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, local_only_params, lexical=True)

cl = trainClassifier(train_data)
print(train_data[0])
y_true = [x[1] for x in test_data]
y_pred_local_only = predictLabels(test_data, cl)
prf = precision_recall_fscore_support(y_true, y_pred_local_only, average='weighted')
print(prf)
print(classification_report(y_true, y_pred_local_only))

In [ ]:
# competitor 3: incrementally trained lexical only

train_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, global_only_params, lexical=True,
                                                                     lm_feats=False)
test_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, global_only_params, lexical=True,
                                                                    lm_feats=False)

y_true = [x[1] for x in test_data]
y_pred_lexical_only_retrain = []

for i in range(0, len(test_data)):
    cl = trainClassifier(train_data)
    y_instance = predictLabels(test_data[i:i+1], cl)
    y_pred_lexical_only_retrain.extend(y_instance)
    train_data.append(test_data[i])    

prf = precision_recall_fscore_support(y_true, y_pred_lexical_only_retrain, average='weighted')
print(prf)
print(classification_report(y_true, y_pred_lexical_only_retrain))

In [ ]:
# competitor 4: incrementally trained lexical + global lm

train_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, global_only_params, lexical=True,
                                                                     lm_feats=True)
test_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, global_only_params, lexical=True,
                                                                    lm_feats=True)

y_true = [x[1] for x in test_data]
y_pred_global_lm_retrain = []

for i in range(0, len(test_data)):
    cl = trainClassifier(train_data)
    y_instance = predictLabels(test_data[i:i+1], cl)
    y_pred_global_lm_retrain.extend(y_instance)
    train_data.append(test_data[i])    

prf = precision_recall_fscore_support(y_true, y_pred_global_lm_retrain, average='weighted')
print(prf)
print(classification_report(y_true, y_pred_global_lm_retrain))

In [ ]:
# competitor 5: incrementally trained lexical + global + local (opt params) lm

train_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, opt_params, lexical=True,
                                                                     lm_feats=True)
test_data = generate_classifier_data_from_global_lm_and_local_params(lm_global, opt_params, lexical=True,
                                                                    lm_feats=True)
y_true = [x[1] for x in test_data]
y_pred_opt_params_retrain = []

for i in range(0, len(test_data)):
    cl = trainClassifier(train_data)
    y_instance = predictLabels(test_data[i:i+1], cl)
    y_pred_opt_params_retrain.extend(y_instance)
    train_data.append(test_data[i])    

prf = precision_recall_fscore_support(y_true, y_pred_opt_params_retrain, average='weighted')
print(prf)
print(classification_report(y_true, y_pred_opt_params_retrain))

In [ ]:
# get significant difference in accuracy scores using macnemar test for p<=0.05, 0.01, 0.001
test_preds_all_models = [y_pred, y_pred_global_only, y_pred_local_only,  y_pred_lexical_only_retrain,
                         y_pred_global_lm_retrain, y_pred_opt_params_retrain]
table_of_comparison = np.zeros(len(test_preds_all_models),len(test_preds_all_models))
# check symetry of results - though only need the triangle without the diagnoal for all against all
for i in range(0, len(test_preds_all_models)):
    for j in range(0,len(test_preds_all_models)):
        cl1_preds = test_preds_all_models[i]
        cl2_preds = test_preds_all_models[j]
        table_of_comparison[i][j] = calculate_mcnemar_test(c11_preds, c12_preds, y_true):
print(table_of_comparison)

In [ ]:
#‘b’	blue
#‘g’	green
#‘r’	red  (255, 0, 0 )
#‘c’	cyan
#‘m’	magenta
#‘y’	yellow
#‘k’	black
#‘w’	white

#orange (255, 128, 0)
#grey (160, 160, 160)
#pink (255, 153, 204)
#brown (153, 76, 0)
#purple (102, 0, 204)

colour_map = {'F': (160/255, 160/255, 160/255), 'I': "blue" , 'L': (255/255, 128/255, 0/255),
              'N': (102/255, 0/255, 204/255),
              'P': (255/255, 153/255, 204/255), 'T': "green", 'U': "yellow", 'V': "blue",
              'W' :"green", 'X': "red", 'Y': (153/255, 76/255, 0/255), 'Z': "blue"}


pair_num = 'r5'
for target in piece_references_and_probabilities_per_pair[pair_num].keys():
    piece_plots = {piece:[] for piece in piece_references_and_probabilities_per_pair.keys()}
    print(target)
    for ref in piece_references_and_probabilities_per_pair[pair_num][target]:
        prob_dict = ref[1]  # 0 is raw probs, 1 is moving average, 
        
        #print(prob_dict)
        for piece in prob_dict.keys():
            #print(piece)
            if piece_plots.get(piece) is None:
                #print("no piece", piece)
                continue
            piece_plots[piece].append(prob_dict[piece])
    #print(piece_plots)

    for piece in sorted(piece_plots.keys()):
        probs = piece_plots[piece]
        target_text = "(TARGET)" if piece == target else ""
        linestyle = "solid" if piece == target else "dashed"
        marker = "o" if piece == target else "none"
        plt.plot(probs, label = piece + " " + target_text, color=colour_map[piece],
                linestyle=linestyle, marker=marker)
 
    
    # plot lines
    #plt.plot(moving_average([m[0] for m in method1]), label = "active: mixed models with k=0.015 for previous")
    #plt.plot(moving_average([m[0] for m in method2]), label = "interactive: retraining language model with incrementing data")
    #plt.plot(moving_average([m[0] for m in method3]), label = "static baseline")
    #plt.plot(moving_average([m[0] for m in method4]), label = "interactive: mixed models with k=0.015 for previous + recency")
    #plt.plot(moving_average([m[0] for m in method5]), label = "active: retraining language model with incrementing data + recency")
    #plt.plot(moving_average([m[0] for m in method6]), label = "interactive only + recency")
    #plt.plot(moving_average([m[0] for m in method7]), label = "interactive only")
    plt.legend(bbox_to_anchor=(0.1, 1))
    plt.show()

In [ ]:
if False:
    predictions_lm_global_lexical = all_preds   ####
    0.8869126862186916
    0.858740835661667
    0.840540883746269
    
if False:
    predictions_local_only = all_preds  
    0.8027383467276631
    0.7081809589292479
    0.7095633954849265
    #### worse than predictions_lm_global_lexical
    ###[[1410, 151], [461, 171]]
    ###statistic=151.000, p-value=0.000
    ###Different proportions of errors (reject H0)
    


if False:
    predictions_local_only_through_model = all_preds
    0.8466507702262193
    0.8220748960978559
    0.8181709249993098
     #### worse than predictions_lm_global_lexical
    # [[1624, 196], [247, 126]]
    # statistic=196.000, p-value=0.017
    # Different proportions of errors (reject H0)
    
    # strangely better than predictions_local_only
    #[[1453, 367], [108, 265]]
    #statistic=108.000, p-value=0.000
    #Different proportions of errors (reject H0)
    
if False:
    predictions_local_with_global_0_1_cutoff_5 = all_preds
    0.9235015228253362
    0.9135418648826275
    0.9033869590431085
    # better than predictions_local_only_through_model
    #[[1710, 110], [281, 92]]
    #statistic=110.000, p-value=0.000
    #Different proportions of errors (reject H0)
    
    # better than predictions_lm_global_lexical
    #get_contingency_table(predictions_local_with_global_0_1_cutoff_5, predictions_lm_global_lexical, all_y)
    #[[1743, 248], [128, 74]]
    #statistic=128.000, p-value=0.000
    #Different proportions of errors (reject H0)

In [ ]:
#!pip install statsmodels
# Example of calculating the mcnemar test
from statsmodels.stats.contingency_tables import mcnemar
# define contingency table


def get_contingency_table(cl1_preds, cl2_preds, ground_truth):
    ##  [[c1Y/c2Y, c1Y/C2N], [c1N/C2Y, c1n/c2n]]
    table = [[0,0], [0,0]]
    for c1, c2, y in zip(cl1_preds, cl2_preds, ground_truth):
        if c1 == y:
            if c2 == y:
                table[0][0]+=1
            else:
                table[0][1]+=1
        else:
            if c2 == y:
                table[1][0]+=1
            else:
                table[1][1]+=1
    return table

c_1 =     ["b", "b", "a", "b", "b", "a", "b", "b","a", "b", "b","a", "b", "b","a", "b"]
c_2 =     ["c", "b", "a", "c", "b", "a", "c", "b","a", "c", "b","a", "c", "b","a", "a"]
y_all =   ["b", "b", "a", "b", "b", "a", "b", "b","a", "b", "b","a", "b", "b","a", "b"]

table = get_contingency_table(c_1, c_2, y_all)
print(table)
# calculate mcnemar test
result = mcnemar(table, exact=True)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
    print('Same proportions of errors (fail to reject H0)')
else:
    print('Different proportions of errors (reject H0)')

In [ ]:
table = get_contingency_table(predictions_local_with_global_0_1_cutoff_5, predictions_lm_global_lexical, all_y)
print(table)
# calculate mcnemar test
result = mcnemar(table, exact=True)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
    print('Same proportions of errors (fail to reject H0)')
else:
    print('Different proportions of errors (reject H0)')

In [ ]:
# Do a cross-fold model by speaker (and get word MI info for referent type)
results = []
piece_counter = Counter()
word_counter = Counter()
word_piece_counter = Counter()

for PAIR_NUM in exclude:
    print(PAIR_NUM)

    train_data = []
    test_data = []

    for piece in piece_references_and_probabilities_per_pair.keys():
        #print(piece)
        if not piece_references_and_probabilities_per_pair.get(piece):
            print("not enough data", piece)
            continue
        for pair_num in piece_references_and_probabilities_per_pair[piece]:
     
            #print(pair_num)
            count_ref_so_far = 0
            for ref_info in piece_references_and_probabilities_per_pair[piece][pair_num]:
                piece_lm_prob_local, piece_lm_moving_average, text, speaker, utt_id, end_time = ref_info
                piece_counter[piece] += 1
                if CORPUS in ["TAKECV", "TAKE"]:
                    local_word_count = Counter()
                    feature_vector = {}
                    for f in db.execute('SELECT word from words WHERE gameID ="' + str(gameID) + '"'):
                        #print(f)
                        for word in clean_utt(f[0].lower()).split():
                            local_word_count[word] += 1
                    word_dem = len(local_word_count.values())
                    for k, v in local_word_count.items():
                        feature_vector[k] = local_word_count[k]/word_dem
                        word_counter[k] += (v/word_dem)
                        word_piece_counter[piece+"__"+k] += (v/word_dem)
                elif CORPUS == "PENTOCV":
                    feature_vector = {}
                    
                    clean_utt = text.lower().split()
                    word_dem = len(clean_utt)
                    
                    local_word_count = Counter()

                    for word in clean_utt:
                        word_counter[word] += (1/word_dem)   # for global
                        word_piece_counter[piece+"__"+word] += (1/word_dem)  #for global
                        local_word_count[word] += 1
                    
                    for k, v in local_word_count.items():
                        feature_vector[k] = local_word_count[k]/word_dem
                    
                    LOCAL = True
                    if LOCAL:
                        #for k, v in piece_lm_prob_local.items():
                        #    feature_vector['local_prob_' + k] = v

                        zscore_dict = get_zscore_dict_for_prob_dict(piece_lm_prob_local)

                        #for k, v in zscore_dict.items():
                        #    feature_vector['local_zscore_prob_' + k] = v 
                        
                        
                        #for k, v in piece_lm_moving_average.items():
                        #    feature_vector['local_prob_moving_' + k] = v
                        
                        
                        zscore_dict_average = get_zscore_dict_for_prob_dict(piece_lm_moving_average)

                        for k, v in zscore_dict_average.items():
                            feature_vector['local_zscore_prob_moving_' + k] = v 
                            
                            
                        #rank_dict = get_rank_dict_for_prob_dict(piece_lm_prob_local)

                        #for k, v in rank_dict.items():
                        #    feature_vector['local_prob_rank' + k] = v 

                    count_ref_so_far+=1
                    if count_ref_so_far > 7:
                        pass
                    #feature_vector["count_ref_so_far"] = count_ref_so_far
                    
                    if pair_num == PAIR_NUM:
                        test_data.append((feature_vector, piece))
                    else:
                        train_data.append((feature_vector, piece))
    cl = trainClassifier(train_data)
    print(train_data[0])
    y_true = [x[1] for x in test_data]
    y_pred = predictLabels(test_data, cl)
    prf = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    results.append(prf)
    print(prf)
    print(classification_report(y_true, y_pred))
print(np.mean([x[0] for x in results]))
print(np.mean([x[1] for x in results]))
print(np.mean([x[2] for x in results]))

In [ ]:
# EXHAUSTIVE RETRAINING BASELINE - TODO NEED TO MAKE IT STRICTLY IN ORDER, WHICH MAKES IT HARDER
results = []
for PAIR_NUM in piece_references_and_probabilities_per_pair['X'].keys():
    print(PAIR_NUM)
    train_data = []
    test_data = []

    for piece in piece_references_and_probabilities_per_pair.keys():
        #print(piece)
        if not piece_references_and_probabilities_per_pair.get(piece):
            print("not enought data", piece)
            continue
        for pair_num in piece_references_and_probabilities_per_pair[piece]:
            #print(pair_num)
            count_ref_so_far = 0
            for ref_info in piece_references_and_probabilities_per_pair[piece][pair_num]:
                piece_lm_prob_local, piece_lm_moving_average, text, speaker, utt_id, end_time = ref_info
                
                if CORPUS in ["TAKECV", "TAKE"]:
                    local_word_count = Counter()
                    feature_vector = {}
                    for f in db.execute('SELECT word from words WHERE gameID ="' + str(gameID) + '"'):
                        #print(f)
                        for word in clean_utt(f[0].lower()).split():
                            local_word_count[word] += 1
                    word_dem = len(local_word_count.values())
                    for k, v in local_word_count.items():
                        feature_vector[k] = local_word_count[k]/word_dem
                        word_counter[k] += (v/word_dem)
                        word_piece_counter[piece+"__"+k] += (v/word_dem)
                elif CORPUS == "PENTOCV":
                    feature_vector = {}
                    
                    clean_utt = text.lower().split()
                    word_dem = len(clean_utt)
                    
                    local_word_count = Counter()

                    for word in clean_utt:
                        #word_counter[word] += (1/word_dem)   # for global
                        #word_piece_counter[piece+"__"+word] += (1/word_dem)  #for global
                        local_word_count[word] += 1
                    
                    for k, v in local_word_count.items():
                        feature_vector[k] = local_word_count[k]/word_dem
                    
                    LOCAL = False
                    if LOCAL:
                        #for k, v in piece_lm_prob_local.items():
                        #    feature_vector['local_prob_' + k] = v

                        zscore_dict = get_zscore_dict_for_prob_dict(piece_lm_prob_local)

                        #for k, v in zscore_dict.items():
                        #    feature_vector['local_zscore_prob_' + k] = v 
                        
                        
                        #for k, v in piece_lm_moving_average.items():
                        #    feature_vector['local_prob_moving_' + k] = v
                        
                        
                        zscore_dict_average = get_zscore_dict_for_prob_dict(piece_lm_moving_average)

                        for k, v in zscore_dict_average.items():
                            feature_vector['local_zscore_prob_moving_' + k] = v 
                            
                            
                        #rank_dict = get_rank_dict_for_prob_dict(piece_lm_prob_local)

                        #for k, v in rank_dict.items():
                        #    feature_vector['local_prob_rank' + k] = v 

                    count_ref_so_far+=1
                    if count_ref_so_far > 7:
                        pass
                    #feature_vector["count_ref_so_far"] = count_ref_so_far
                    
                    if pair_num == PAIR_NUM:
                        test_data.append((feature_vector, piece))
                    else:
                        train_data.append((feature_vector, piece))
    
    y_true = [x[1] for x in test_data]
    y_pred = []
    # do exhaustive incremental testing on current exmaple then retraining given last example
    for t in range(0, len(test_data)):
        cl = trainClassifier(train_data)
        #print(train_data[0])
    
        y_pred.extend(predictLabels([test_data[t]], cl))
        feats, label = test_data[t]
        feats = {k:v for k, v in feats.items()}
        train_data.append((feats, label))
    prf = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    results.append(prf)
    print(prf)
    print(classification_report(y_true, y_pred))
print(np.mean([x[0] for x in results]))
print(np.mean([x[1] for x in results]))
print(np.mean([x[2] for x in results]))


In [ ]:
# first 7
0.799084599407264
0.7763501742160279
0.7687179024923494

# 7th reference onwards
0.9070414560007611
0.8860422229872891
0.8913018153266432

######
# no local: first 7th
0.8081007772106106
0.7847827138985676
0.769680836038908


# 7th reference onwards
0.8870381606486294
0.8416727513847977
0.8452945489474868


######## ANAPHORS INCLUDED

# Just lexical (no local):
folds = [(0.8343362282878413, 0.6696428571428571, 0.6646344339585925, None), (0.7751388356942908, 0.6567796610169492, 0.6389782650087881, None), (0.8436337379459808, 0.8318840579710145, 0.8288104055372693, None), (0.8113474218938344, 0.748730964467005, 0.7500110474092568, None), (0.8006385394394444, 0.7194570135746606, 0.7139708353420309, None), (0.8865255347897608, 0.8523985239852399, 0.8480169565052978, None), (0.875462715233778, 0.8433420365535248, 0.8458301462869497, None), (0.6828855495800346, 0.5995717344753747, 0.5785474667579829, None)] + probs
0.8137460703581206
0.7402258561483283
0.733599944600771

# NB also get this? (2)
0.7760773490842137
0.6742929646244494
0.6870837082841104



# Lexical + large language model only zscore + moving average zscore:
0.8783176188823876
0.864797219798711
0.8477371357172677

# Lexical + local only
0.8512635887308037
0.8056822726271888
0.804419210894042

# (NB THIS CAN BE WORSE IN THE NON-COMBINATION MODEL AT 0.73?)

# Lexical + global + local mix (0.1 and 5 word cut-off)
0.9454755299899154
0.9384068603538054
0.9326283115477803



# RETRAINING AFTER EVERY STEP WITH LOCAL MODEL FEATURES
0.9743351380668108
0.9705121847833994
0.9706865125416868

# RETRAINING AFTER EVERY STEP WITH NON LOCAL LM FEATURES
0.9648312888445624
0.959002612053766
0.9585806352758561

# RETRAINING AFTER EVERY STEP WITH NO LOCAL MODEL FEATURES (JUST LEXICAL) - see (2 above)
0.7726613998556959
0.7074927259648329
0.7179627212445062

In [ ]:
print(results, "+ probs")
print(np.mean([x[0] for x in results]))
print(np.mean([x[1] for x in results]))
print(np.mean([x[2] for x in results]))

print(results, "+ zscores, no probs")
print(np.mean([x[0] for x in results]))
print(np.mean([x[1] for x in results]))
print(np.mean([x[2] for x in results]))
Training Classifier...
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
(0.8643671328671328, 0.82, 0.8271238761238762, None)
              precision    recall  f1-score   support

           F       1.00      1.00      1.00         5
           I       0.57      0.57      0.57         7
           L       1.00      0.83      0.91        12
           N       0.62      0.83      0.71         6
           P       1.00      0.80      0.89        15
           T       1.00      0.86      0.92        14
           U       0.80      0.50      0.62         8
           V       0.33      0.67      0.44         3
           W       0.62      1.00      0.76         8
           X       0.82      1.00      0.90         9
           Y       1.00      0.91      0.95        11
           Z       1.00      0.50      0.67         2

    accuracy                           0.82       100
   macro avg       0.81      0.79      0.78       100
weighted avg       0.86      0.82      0.83       100

Training Classifier...
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
(0.8240871203100304, 0.8176470588235294, 0.8150765126290163, None)
              precision    recall  f1-score   support

           F       0.92      0.69      0.79        16
           I       0.62      0.57      0.59        14
           L       0.80      0.62      0.70        13
           N       1.00      0.93      0.96        14
           P       0.82      1.00      0.90         9
           T       0.94      0.94      0.94        16
           U       0.73      1.00      0.85        11
           V       0.67      0.57      0.62         7
           W       0.91      0.95      0.93        21
           X       0.84      0.94      0.89        17
           Y       0.64      0.74      0.68        19
           Z       0.91      0.77      0.83        13

    accuracy                           0.82       170
   macro avg       0.82      0.81      0.81       170
weighted avg       0.82      0.82      0.82       170

Training Classifier...
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
(0.9174162912751319, 0.9060150375939849, 0.9048130247138406, None)
              precision    recall  f1-score   support

           F       1.00      0.94      0.97        18
           I       0.70      0.95      0.81        20
           L       0.88      1.00      0.94        22
           N       1.00      0.96      0.98        24
           P       0.89      0.89      0.89        18
           T       0.94      0.85      0.89        34
           U       1.00      1.00      1.00        18
           V       0.79      0.85      0.81        26
           W       0.82      0.90      0.86        20
           X       1.00      1.00      1.00        26
           Y       1.00      1.00      1.00        19
           Z       1.00      0.57      0.73        21

    accuracy                           0.91       266
   macro avg       0.92      0.91      0.91       266
weighted avg       0.92      0.91      0.90       266

Training Classifier...
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
(0.8877626562741163, 0.8807017543859649, 0.8826880082677435, None)
              precision    recall  f1-score   support

           F       1.00      0.94      0.97        18
           I       0.72      0.86      0.78        21
           L       0.62      0.83      0.71        12
           N       0.90      0.90      0.90        29
           P       0.85      0.85      0.85        20
           T       0.92      0.85      0.88        26
           U       0.97      0.94      0.95        33
           V       0.87      0.84      0.85        31
           W       0.91      0.87      0.89        23
           X       0.94      0.97      0.95        30
           Y       1.00      0.92      0.96        25
           Z       0.75      0.71      0.73        17

    accuracy                           0.88       285
   macro avg       0.87      0.87      0.87       285
weighted avg       0.89      0.88      0.88       285

Training Classifier...
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
(0.8661624951217711, 0.8531468531468531, 0.8533600723286038, None)
              precision    recall  f1-score   support

           F       0.86      0.86      0.86         7
           I       0.33      0.20      0.25         5
           L       0.65      1.00      0.79        11
           N       1.00      0.87      0.93        15
           P       1.00      1.00      1.00        12
           T       0.75      0.90      0.82        10
           U       1.00      1.00      1.00        15
           V       0.46      0.55      0.50        11
           W       1.00      0.86      0.92        14
           X       1.00      1.00      1.00        16
           Y       1.00      0.88      0.94        17
           Z       0.75      0.60      0.67        10

    accuracy                           0.85       143
   macro avg       0.82      0.81      0.81       143
weighted avg       0.87      0.85      0.85       143

Training Classifier...
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
(0.8492155043476253, 0.8178294573643411, 0.8104505814768325, None)
              precision    recall  f1-score   support

           F       0.94      0.94      0.94        17
           I       0.81      0.65      0.72        26
           L       0.88      0.84      0.86        25
           N       0.97      1.00      0.99        33
           P       0.96      1.00      0.98        26
           T       1.00      0.38      0.56        26
           U       1.00      0.94      0.97        17
           V       0.67      0.45      0.54        22
           W       0.58      0.91      0.71        23
           X       0.92      1.00      0.96        12
           Y       0.76      0.96      0.85        23
           Z       0.44      0.88      0.58         8

    accuracy                           0.82       258
   macro avg       0.83      0.83      0.80       258
weighted avg       0.85      0.82      0.81       258

Training Classifier...
/Users/julianhough/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
(0.877119243542806, 0.8636363636363636, 0.8635317127546905, None)
              precision    recall  f1-score   support

           F       1.00      0.90      0.95        29
           I       0.54      0.83      0.66        30
           L       1.00      0.86      0.92        35
           N       0.98      1.00      0.99        41
           P       0.96      1.00      0.98        25
           T       0.83      0.80      0.82        25
           U       0.92      1.00      0.96        23
           V       0.71      0.67      0.69        33
           W       0.77      0.82      0.79        28
           X       1.00      0.97      0.99        38
           Y       1.00      1.00      1.00        23
           Z       0.75      0.41      0.53        22

    accuracy                           0.86       352
   macro avg       0.87      0.85      0.86       352
weighted avg       0.88      0.86      0.86       352

Training Classifier...
(0.7745976880211548, 0.7553516819571865, 0.7516840670381353, None)
              precision    recall  f1-score   support

           F       0.75      0.60      0.67        25
           I       0.55      0.38      0.44        16
           L       0.69      0.87      0.77        31
           N       0.85      0.49      0.62        35
           P       0.94      0.77      0.85        22
           T       0.80      0.90      0.85        40
           U       0.95      0.95      0.95        19
           V       0.64      0.62      0.63        29
           W       0.70      0.85      0.77        33
           X       0.93      0.83      0.88        30
           Y       0.91      0.80      0.85        25
           Z       0.53      0.91      0.67        22

    accuracy                           0.76       327
   macro avg       0.77      0.75      0.74       327
weighted avg       0.77      0.76      0.75       327

0.857591016469971
0.839291025863528
0.8385909819165923

In [ ]:
print(results, "no local probs")
print(np.mean([x[0] for x in results]))
print(np.mean([x[1] for x in results]))
print(np.mean([x[2] for x in results]))

Training Classifier...
(0.8174500891265598, 0.72, 0.7050823983034509, None)
              precision    recall  f1-score   support

           F       1.00      1.00      1.00         5
           I       0.42      0.71      0.53         7
           L       0.62      0.83      0.71        12
           N       1.00      0.67      0.80         6
           P       1.00      0.20      0.33        15
           T       1.00      0.86      0.92        14
           U       0.75      0.38      0.50         8
           V       0.40      0.67      0.50         3
           W       0.47      1.00      0.64         8
           X       0.82      1.00      0.90         9
           Y       1.00      0.91      0.95        11
           Z       1.00      0.50      0.67         2

    accuracy                           0.72       100
   macro avg       0.79      0.73      0.70       100
weighted avg       0.82      0.72      0.71       100

Training Classifier...
(0.834859943977591, 0.8117647058823529, 0.8149470677453468, None)
              precision    recall  f1-score   support

           F       0.73      0.69      0.71        16
           I       0.60      0.64      0.62        14
           L       0.60      0.69      0.64        13
           N       1.00      0.93      0.96        14
           P       0.75      1.00      0.86         9
           T       1.00      0.94      0.97        16
           U       0.79      1.00      0.88        11
           V       0.80      0.57      0.67         7
           W       0.67      0.86      0.75        21
           X       1.00      0.88      0.94        17
           Y       1.00      0.74      0.85        19
           Z       1.00      0.77      0.87        13

    accuracy                           0.81       170
   macro avg       0.83      0.81      0.81       170
weighted avg       0.83      0.81      0.81       170

Training Classifier...
(0.9159195345407927, 0.9022556390977443, 0.9025895692452572, None)
              precision    recall  f1-score   support

           F       1.00      0.89      0.94        18
           I       0.63      0.95      0.76        20
           L       0.96      1.00      0.98        22
           N       1.00      0.96      0.98        24
           P       0.94      0.89      0.91        18
           T       0.94      0.85      0.89        34
           U       1.00      1.00      1.00        18
           V       0.79      0.85      0.81        26
           W       0.82      0.90      0.86        20
           X       1.00      1.00      1.00        26
           Y       1.00      1.00      1.00        19
           Z       0.92      0.57      0.71        21

    accuracy                           0.90       266
   macro avg       0.92      0.90      0.90       266
weighted avg       0.92      0.90      0.90       266

Training Classifier...
(0.8976078088578088, 0.887719298245614, 0.8902415421467847, None)
              precision    recall  f1-score   support

           F       0.94      0.94      0.94        18
           I       0.68      0.90      0.78        21
           L       0.67      0.83      0.74        12
           N       1.00      0.86      0.93        29
           P       0.85      0.85      0.85        20
           T       0.92      0.85      0.88        26
           U       0.97      0.94      0.95        33
           V       0.87      0.84      0.85        31
           W       0.91      0.87      0.89        23
           X       1.00      0.97      0.98        30
           Y       0.88      0.92      0.90        25
           Z       0.88      0.82      0.85        17

    accuracy                           0.89       285
   macro avg       0.88      0.88      0.88       285
weighted avg       0.90      0.89      0.89       285

Training Classifier...
(0.8651029486076791, 0.8531468531468531, 0.8536309338434843, None)
              precision    recall  f1-score   support

           F       0.86      0.86      0.86         7
           I       0.33      0.20      0.25         5
           L       0.65      1.00      0.79        11
           N       1.00      0.87      0.93        15
           P       1.00      1.00      1.00        12
           T       0.82      0.90      0.86        10
           U       1.00      1.00      1.00        15
           V       0.46      0.55      0.50        11
           W       1.00      0.86      0.92        14
           X       1.00      1.00      1.00        16
           Y       1.00      0.88      0.94        17
           Z       0.67      0.60      0.63        10

    accuracy                           0.85       143
   macro avg       0.82      0.81      0.81       143
weighted avg       0.87      0.85      0.85       143

Training Classifier...
(0.8656913221242073, 0.8333333333333334, 0.834707644846363, None)
              precision    recall  f1-score   support

           F       1.00      0.94      0.97        17
           I       0.65      0.65      0.65        26
           L       1.00      0.84      0.91        25
           N       0.97      1.00      0.99        33
           P       1.00      1.00      1.00        26
           T       1.00      0.62      0.76        26
           U       1.00      0.94      0.97        17
           V       0.62      0.36      0.46        22
           W       0.51      0.91      0.66        23
           X       1.00      1.00      1.00        12
           Y       0.96      0.96      0.96        23
           Z       0.50      0.88      0.64         8

    accuracy                           0.83       258
   macro avg       0.85      0.84      0.83       258
weighted avg       0.87      0.83      0.83       258

Training Classifier...
(0.884811507936508, 0.8664772727272727, 0.8651566296235648, None)
              precision    recall  f1-score   support

           F       1.00      0.90      0.95        29
           I       0.67      0.87      0.75        30
           L       1.00      0.86      0.92        35
           N       0.98      1.00      0.99        41
           P       1.00      1.00      1.00        25
           T       0.83      0.80      0.82        25
           U       0.92      1.00      0.96        23
           V       0.59      0.79      0.68        33
           W       0.77      0.82      0.79        28
           X       0.97      0.89      0.93        38
           Y       1.00      1.00      1.00        23
           Z       0.89      0.36      0.52        22

    accuracy                           0.87       352
   macro avg       0.88      0.86      0.86       352
weighted avg       0.88      0.87      0.87       352

Training Classifier...
(0.8038244434453188, 0.7798165137614679, 0.7780526830735471, None)
              precision    recall  f1-score   support

           F       0.69      0.72      0.71        25
           I       0.44      0.44      0.44        16
           L       0.84      0.87      0.86        31
           N       0.94      0.49      0.64        35
           P       0.94      0.73      0.82        22
           T       0.86      0.90      0.88        40
           U       1.00      0.95      0.97        19
           V       0.70      0.66      0.68        29
           W       0.69      0.94      0.79        33
           X       0.86      0.80      0.83        30
           Y       0.96      0.88      0.92        25
           Z       0.57      0.91      0.70        22

    accuracy                           0.78       327
   macro avg       0.79      0.77      0.77       327
weighted avg       0.80      0.78      0.78       327

0.8606584498270582
0.8318142020243298
0.8305510586034749

In [ ]:
if False:
    #10-f cross_val on individual pairs (smaller datasets)
    ind_results= {"r1" : (0.8224358974358974, 0.7846153846153846, 0.7778205128205128),
    "r2" : (0.7678858784893269, 0.6637931034482759, 0.675242522256069),
    "r3" : (0.8181915306915307, 0.7599067599067599, 0.758845497936407),
    "r4" : (0.7992240687681375, 0.68470066518847, 0.6877442020358089),
    "r5" : (0.7639380411255411, 0.6191666666666666, 0.6379182692307691),
    "r6" : (0.9057487922705313, 0.8761528326745719, 0.8737389495294634),
    "r7" : (0.8732662337662337, 0.8296739130434782, 0.8278420650668934),
    "r8" : (0.7130039355485784, 0.5734432234432234, 0.5872288333290527)}

    print("Trained on same data 10f cross (average performance per pair) av 1/8 of data")
    print(np.mean([x[0] for x in ind_results.values()]))
    print(np.mean([x[1] for x in ind_results.values()]))
    print(np.mean([x[2] for x in ind_results.values()]))

    print("*" * 30)
    print("Trained on all data (mixed) 10-fold cross val (of random 1/8th of data)")
    #10-f cross_val mixed overall (whole dataset)
    global_result_10f_of_random_eighth = [
        (0.7303828289936665, 0.6563546798029556, 0.6591810707081643),
        (0.7773714129329894, 0.7020689655172414, 0.705385763114827),
        (0.7340298765274135, 0.6626600985221675, 0.6634152186073369),
        (0.763595589960122, 0.6799014778325124, 0.6800262533435527)
    ]
    print(np.mean([x[0] for x in global_result_10f_of_random_eighth]))
    print(np.mean([x[1] for x in global_result_10f_of_random_eighth]))
    print(np.mean([x[2] for x in global_result_10f_of_random_eighth]))


    print("*" * 30)
    print("Trained on all data (mixed) 8-fold cross val (full_data)")
    #10-f cross_val mixed overall (whole dataset)
    global_result_8 = (0.7844808924585374, 0.7293746248177686, 0.7410059116896628)
    global_result_10 = (0.7905361390798162, 0.7292899089488714, 0.7417777879399795)
    for x in global_result_8:
        print(x)
    

print("*" * 30)
#mixed- trained on other 7
other_results= {"r1" : (0.7779836356055868, 0.6829268292682927, 0.690424865840207, None),
"r2" : (0.7219102027091829, 0.6401384083044983, 0.6505178871728421, None),
"r3" : (0.8292961148443981, 0.734375, 0.7530287500141206, None),
"r4" : (0.7893900944674533, 0.7048054919908466, 0.7258770742493045, None),
"r5" : (0.7215635412139649, 0.6059322033898306, 0.6263491634014793, None),
"r6" : (0.8328990107115107, 0.809375, 0.8035774233470034, None),
"r7" : (0.8375586804425542, 0.8149779735682819, 0.818528077822021, None),
"r8" : (0.7323545548007373, 0.5745098039215686, 0.5976834631543789, None)}

print("Trained on other data 8-dialogue-fold cross-val")
print(np.mean([x[0] for x in other_results.values()]))
print(np.mean([x[1] for x in other_results.values()]))
print(np.mean([x[2] for x in other_results.values()]))

In [ ]:
good_pieces = ["X", "Y", "P", "N", "U", "F", "Z", "L", "T", "I", "W", "V", "UNK"]
print("non standard pieces", {k:v for k,v in piece_counter.items() if k not in good_pieces})
piece_counter

In [ ]:
print(len(word_counter), "words")
word_counter.most_common(20)

In [ ]:
word_total = sum(word_piece_counter.values())
piece_total= sum(piece_counter.values())

In [ ]:

for piece, piece_count in sorted(piece_counter.items()):
    p_piece = piece_count/piece_total
    print("piece:", piece, piece_count, p_piece)
    highest = -1
    best_word = ""
    rank = {}
    for word, w_count in word_counter.items():
        #print(word, w_count)
        if w_count < 2: 
            continue
        p_word = w_count / word_total
        p_word_piece = word_piece_counter[piece+"__"+word] / word_total
        #check bayes = p(w|c)p(c) = p(c|w)p(w)
        bayes_check_1 = (word_piece_counter[piece+"__"+word]/piece_count) * p_piece
        bayes_check_2 = (word_piece_counter[piece+"__"+word]/w_count) * p_word
        if not round(bayes_check_1,4)== round(bayes_check_2,4):
            print("MISMATCH", word, bayes_check_1, bayes_check_2)
        mi = (p_word_piece/(p_piece * p_word))
        rank[word] = (mi, word_piece_counter[piece+"__"+word], p_word_piece, w_count, p_word)
        if mi > highest:
            highest = mi
            best_word = word
    if True:
        top = 30
        for k, v in sorted(rank.items(), key=lambda x:x[1][0], reverse=True):
            print(k,v[0], [round(x,4) for x in v[1:]])
            top -=1
            if top <= 0: 
                break
    print("*" * 30)

In [ ]:
colour_map = {'F': "grey", 'I': "dark blue" , 'L': "orange", 'N': "purple",
              'P': "pink", 'T': "dark green", 'U': "yellow", 'V': "blue",
              'W' :"green", 'X': "red", 'Y': "brown", 'Z': "blue"}


In [ ]:
db.close()

In [ ]:
"""
p(w, c) = p(w|c)p(c) = p(c|w)p(w)
p(w) is the liklihood of the word appearing in a referring expression
p(c) is the liklihood of a referring expression having this class
p(w|c) is the liklihood of a word appearing given we've got this class
p(c|w) is the liklihood of this class appearing given we've got this word
"""